### To do list: 

create error messages\
include trys for every link\
remake old scrapers\
Ask Sascha about:\

-make savestates\

### Issues: 
The CBDVITAL code currently requires a new instantce of the webdriver every time, because if left open the code returned after the first opening is not the same as seen in the webbrowser, the new instances slow down the program.\

# Webscraper

## Setup
In this Part the necessary libraries are imported, as well as the URLs for the websites.

### Open Window

In [1]:
import tkinter as tk
import threading
import sys
import os
from tkinter import ttk
import time
import math
import sys

def get_status(scraper_number):
    increments = 100/29    
    increments = math.floor(increments) #this will stop working if scraper number > 100
    status = scraper_number * increments
    return status

class App(threading.Thread):

    def __init__(self, status):
        threading.Thread.__init__(self)
        self.status = status
        self.start()

    def callback(self):        
        self.root.quit()
        #os._exit(1)
        
    def update_progress_label(self):
            return f"Current Progress: {self.Progress_Bar['value']}%"
                        
    def task(self):
        if self.Progress_Bar['value'] < 100:
            
            try:                
                self.Progress_Bar['value'] = self.status
                self.value_label['text'] = self.update_progress_label()
            except:
                a = 1
        else:
            self.running_text = ttk.Label(self.root, text="Process Completed.\nThe files have been saved in the same directory as the Webscraper.exe\nYou can now click stop.", font = "Arial 14")
            self.running_text.grid(column=0, row=0, columnspan=2, padx=10, pady=20)
        self.root.after(2000, self.task)  # reschedule event in 2 seconds

    def run(self):
        self.root = tk.Tk()
        self.root.protocol("WM_DELETE_WINDOW", self.callback)
        self.root.title('Webscraper by Lukas Link')
        self.running_text = ttk.Label(self.root, text="The programm is running. This will take at least 10 minutes.", font = "Arial 14")
        self.running_text.grid(column=0, row=0, columnspan=2, padx=10, pady=20)
        self.Progress_Bar=ttk.Progressbar(self.root,orient=tk.HORIZONTAL,length=550,mode='determinate')
        self.Progress_Bar.grid(column=0, row=2, columnspan=2, padx=10, pady=20)
        
        
        # label
        self.value_label = ttk.Label(self.root, text=self.update_progress_label())
        self.value_label.grid(column=0, row=1, columnspan=2)
       
                
               
        # stop button
        stop_button = ttk.Button(
            self.root,
            text='Stop',
            command = self.callback
        )
        stop_button.grid(row=3, padx=250, pady=10, sticky=tk.W)
        
       
        

        self.root.after(2000, self.task)
        
        self.root.mainloop()


app = App(0)
print('Now we can continue running code while mainloop runs!')

Now we can continue running code while mainloop runs!


### Module Import

In [2]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
from forex_python.converter import CurrencyRates # for currency exchange rate
cur = CurrencyRates()
import re #for extracting strings in brackets
import pathlib
import time
start = time.time()
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common import keys
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

#set the options to run Edge in Headless mode.
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
options = EdgeOptions()
options.use_chromium = True
options.add_argument("headless")
options.add_argument("disable-gpu")
options.add_argument('window-size=1920x1080')

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
           "Accept-Encoding":"gzip, deflate", 
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
           "DNT":"1","Connection":"close", 
           "Upgrade-Insecure-Requests":"1"}
app.status = 1

### URL import

In [3]:
#New URL import
path= pathlib.Path().resolve()
path = str(path) + "/URLs.xlsx"
URL_df = pd.DataFrame(pd.read_excel(path))

URL_df = URL_df.dropna() # Remove empty rows
URL_df = URL_df.reset_index(drop=True) # reset index

### Functions

#### edge_path

In [4]:
def edge_path ():
    path= pathlib.Path().resolve()
    path = str(path) + "/msedgedriver.exe"
    return path

#### all_equal

In [5]:
#Function for checking if all numbers are equal   
def all_equal(l):
    if(len(l) < 0):
        r = True
    r = all(ele == l[0] for ele in l)

    if(r):
        return True

    else:
        return False
  

#### Converter

In [6]:
  
#function for currency exchanges
def converter (source, money):
    money = cur.convert(source ,"EUR", money)
    money = round(money, 3)
    return money

#### round_to_2 (decimals after comma)

In [7]:
#function that rounds to to letters ignoring 0 after dezimal
def round_to_2 (number):
    count = 0
    while number < 1:
        count = count + 1
        number = number * 10
    number = round(number, 2)
    number = number / 10 ** count
    
    return number

#### skip_if_no_oil

In [8]:
def skip_if_no_oil (name, description, multiple_options = False):
    if multiple_options == True:
        skip = False
        for j in range(len(description)):
            
            if len(re.findall("Capsules", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Kapsel", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Kapsel", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Capsules", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Balm", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Balm", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("BÁLSAMO", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("BÁLSAMO", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Wax", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Wax", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Salve", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Salve", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Salbe", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("Salbe", description[j], re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("cbg", name, re.IGNORECASE)) > 0:
                skip = True
            if len(re.findall("cbg", description[j], re.IGNORECASE)) > 0:
                skip = True
    else:
        skip = False
        if len(re.findall("Capsules", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Kapsel", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Kapsel", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Capsules", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Balm", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Balm", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("BÁLSAMO", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("BÁLSAMO", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Wax", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Wax", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Salve", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Salve", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Salbe", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("Salbe", description, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("cbg", name, re.IGNORECASE)) > 0:
            skip = True
        if len(re.findall("cbg", description, re.IGNORECASE)) > 0:
            skip = True
    
    return skip 

#### get_mg_cbd

In [9]:
def get_mg_cbd(name, description, multiple_options = False, option_s = [] ):
    
    mg_cbd_exists = False
    
    if multiple_options == True:
        try:
            mg_cbd_s = []
            mg_cbd_exists = True 
            for j in range(len(option_s)):
                mg_cbd = name.replace(" ","")
                mg_cbd = mg_cbd.replace(".","")
                check = mg_cbd
                mg_cbd = re.findall(r"(\d\.\d|\d+)mg", mg_cbd, re.IGNORECASE)
                mg_cbd = [float(l) for l in mg_cbd]
                mg_cbd = [x for x in mg_cbd if x > 150]
                #check if there are multiple values in the style of: 5-25%
                if len(re.findall(r"(\d+-\d+)mg", check, re.IGNORECASE)) > 0:
                    mg_cbd = []
                #we use the mg_cbd we found if there is only one, and that one is smaller than 50%
                if (((len(mg_cbd) == 1) | (all_equal(mg_cbd) & (len(mg_cbd) != 0)))):
                    mg_cbd = mg_cbd[0]
                    #print("if")
                else:
                    #print("else")
                    mg_cbd = description[j].replace(".","")
                    check = mg_cbd
                    mg_cbd = re.findall(r"(\d\.\d|\d+)mg", mg_cbd, re.IGNORECASE)
                    mg_cbd = [float(l) for l in mg_cbd]
                    mg_cbd = [x for x in mg_cbd if x > 150]
                    #check if there are multiple values in the style of: 5-25%
                    if len(re.findall(r"(\d+-\d+)mg", check, re.IGNORECASE)) > 0:
                        mg_cbd = []
                    #Check if there are different % numbers, in which case we can not automatically determin the correct one.
                    if (((len(mg_cbd) == 1) | (all_equal(mg_cbd) & (len(mg_cbd) != 0)))):
                        mg_cbd = mg_cbd[0]
                        #print("else-if")
                    else:
                        mg_cbd = []
                        mg_cbd_exists = False
                        #print("else-else")
                        
                mg_cbd_s.append(mg_cbd)
                
            mg_cbd = mg_cbd_s
        except Exception as e:
            print("get_mg_cbd-multiple raises error:")
            print(e)
            mg_cbd = []
            mg_cbd_exists = False  
    else:
        try:
            mg_cbd = name.replace(" ","")
            mg_cbd = mg_cbd.replace(".","")
            check = mg_cbd
            mg_cbd = re.findall(r"(\d\.\d|\d+)mg", mg_cbd, re.IGNORECASE)
            mg_cbd = [float(l) for l in mg_cbd]
            mg_cbd = [x for x in mg_cbd if x > 150]
            #check if there are multiple values in the style of: 5-25%
            if len(re.findall(r"(\d+-\d+)mg", check, re.IGNORECASE)) > 0:
                mg_cbd = []
            #we use the mg_cbd we found if there is only one, and that one is smaller than 50%
            if (((len(mg_cbd) == 1) | (all_equal(mg_cbd) & (len(mg_cbd) != 0)))):
                mg_cbd = mg_cbd[0]
                mg_cbd_exists = True
            else:
                mg_cbd = description.replace(".","")
                check = mg_cbd
                mg_cbd = re.findall(r"(\d\.\d|\d+)mg", mg_cbd, re.IGNORECASE)
                mg_cbd = [float(l) for l in mg_cbd]
                mg_cbd = [x for x in mg_cbd if x > 150]
                #check if there are multiple values in the style of: 5-25%
                if len(re.findall(r"(\d+-\d+)mg", check, re.IGNORECASE)) > 0:
                    mg_cbd = []
                #Check if there are different % numbers, in which case we can not automatically determin the correct one.
                if (((len(mg_cbd) == 1) | (all_equal(mg_cbd) & (len(mg_cbd) != 0)))):
                    mg_cbd = mg_cbd[0]
                    mg_cbd_exists = True
                else:
                    mg_cbd = []
                    mg_cbd_exists = False
        except Exception as e:
            print("get_mg_cbd-single raises error:")
            print(e)
            mg_cbd = []
            mg_cbd_exists = False
            
    return mg_cbd, mg_cbd_exists

#### get_percent_cbd

In [10]:
def get_percent_cbd(name, description, multiple_options = False, option_s = [] ):
    
    percent_exists = False
    
    if multiple_options == True:
        try:
            percent_cbd_s = []
            percent_exists = True 
            for j in range(len(option_s)):
                percent_cbd = name.replace(" ","")
                percent_cbd = percent_cbd.replace(",",".")
                check = percent_cbd.replace(".","")
                percent_cbd = re.findall(r"(\d+\.\d+|\d+)%", percent_cbd)
                percent_cbd = [float(l) for l in percent_cbd]
                percent_cbd = [x for x in percent_cbd if x < 50]
                #check if there are multiple values in the style of: 5-25%
                if len(re.findall(r"(\d+-\d+)%", check, re.IGNORECASE)) > 0:
                    percent_cbd = []
                #we use the percentage we found if there is only one, and that one is smaller than 50%
                if (((len(percent_cbd) == 1) | (all_equal(percent_cbd) & (len(percent_cbd) != 0)))):
                    percent_cbd = percent_cbd[0]
                else:            
                    percent_cbd = description[j].replace(",",".")
                    check = percent_cbd.replace(".","")
                    percent_cbd = re.findall(r"(\d+\.\d+|\d+)%", percent_cbd)
                    percent_cbd = [float(l) for l in percent_cbd]
                    percent_cbd = [x for x in percent_cbd if x < 50]
                    #check if there are multiple values in the style of: 5-25%
                    if len(re.findall(r"(\d+-\d+)%", check, re.IGNORECASE)) > 0:
                        percent_cbd = []
                    #Check if there are different % numbers, in which case we can not automatically determin the correct one.
                    if (((len(percent_cbd) == 1) | (all_equal(percent_cbd) & (len(percent_cbd) != 0)))):
                        percent_cbd = percent_cbd[0]
                    else: 
                        percent_cbd = []
                        percent_exists = False
                        
                percent_cbd_s.append(percent_cbd)
                
            percent_cbd = percent_cbd_s
        except Exception as e:
            print("get_percent_cbd-multiple raises error:")
            print(e)
            percent_cbd = []
            percent_exists = False  
    else:
        try:
            percent_cbd = name.replace(" ","")
            percent_cbd = percent_cbd.replace(",",".")
            check = percent_cbd.replace(".","")
            percent_cbd = re.findall(r"(\d+\.\d+|\d+)%", percent_cbd)
            percent_cbd = [float(l) for l in percent_cbd]
            percent_cbd = [x for x in percent_cbd if x < 50]
            #check if there are multiple values in the style of: 5-25%
            if len(re.findall(r"(\d+-\d+)%", check, re.IGNORECASE)) > 0:
                percent_cbd = []
            #we use the percentage we found if there is only one, and that one is smaller than 50%
            if (((len(percent_cbd) == 1) | (all_equal(percent_cbd) & (len(percent_cbd) != 0)))):
                percent_cbd = percent_cbd[0]
                percent_exists = True
            else:            
                percent_cbd = description.replace(",",".")
                check = percent_cbd.replace(".","")
                percent_cbd = re.findall(r"(\d+\.\d+|\d+)%", percent_cbd)
                percent_cbd = [float(l) for l in percent_cbd]
                percent_cbd = [x for x in percent_cbd if x < 50]
                #check if there are multiple values in the style of: 5-25%
                if len(re.findall(r"(\d+-\d+)%", check, re.IGNORECASE)) > 0:
                    percent_cbd = []
                #Check if there are different % numbers, in which case we can not automatically determin the correct one.
                if (((len(percent_cbd) == 1) | (all_equal(percent_cbd) & (len(percent_cbd) != 0)))):
                    percent_cbd = percent_cbd[0]
                    percent_exists = True
                else: 
                    percent_cbd = []
                    percent_exists = False
        except Exception as e:
            print("get_percent_cbd-single raises error:")
            print(e)
            percent_cbd = []
            percent_exists = False
            
    return percent_cbd, percent_exists

#### get_volume_ml

In [11]:
def get_volume_ml(name, description, multiple_options = False, option_s = [] ):
    
    volume_exists = False
    
    if multiple_options == True:
        try:
            volume_ml_s = []
            volume_exists = True 
            for j in range(len(option_s)):
                volume_ml = name.replace(" ","")
                volume_ml = volume_ml.replace(",",".")
                check = volume_ml.replace(".","")
                volume_ml = re.findall(r"(\d+\.\d+|\d+)ml", volume_ml, re.IGNORECASE)
                volume_ml = [float(l) for l in volume_ml]
                volume_ml = [x for x in volume_ml if x <= 300]
                #check if there are multiple values in the style of: 5-25%
                if len(re.findall(r"(\d+-\d+)ml", check, re.IGNORECASE)) > 0:
                    volume_ml = []
                #we use the volume_ml we found if there is only one
                if (((len(volume_ml) == 1) | (all_equal(volume_ml) & (len(volume_ml) != 0)))):
                    volume_ml = volume_ml[0]
                else:
                    volume_ml = description[j].replace(",",".")
                    check = volume_ml.replace(".","")
                    volume_ml = re.findall(r"(\d+\.\d+|\d+)ml", volume_ml, re.IGNORECASE)
                    volume_ml = [float(l) for l in volume_ml]
                    volume_ml = [x for x in volume_ml if x <= 300]
                    #check if there are multiple values in the style of: 5-25%
                    if len(re.findall(r"(\d+-\d+)ml", check, re.IGNORECASE)) > 0:
                        volume_ml = []
                    #Check if there are different ml numbers, in which case we can not automatically determin the correct one.
                    if (((len(volume_ml) == 1) | (all_equal(volume_ml) & (len(volume_ml) != 0)))):
                        volume_ml = volume_ml[0]
                    else: 
                        volume_ml = []
                        volume_exists = False
                volume_ml_s.append(volume_ml)
            volume_ml = volume_ml_s
        except Exception as e:
            print("get_volume_ml-multiple raises error:")
            print(e)
            volume_ml = []
            volume_exists = False  
    else:
        try:
            volume_ml = name.replace(" ","")
            volume_ml = volume_ml.replace(",",".")
            check = volume_ml.replace(".","")
            volume_ml = re.findall(r"(\d+\.\d+|\d+)ml", volume_ml, re.IGNORECASE)
            volume_ml = [float(l) for l in volume_ml]
            volume_ml = [x for x in volume_ml if x <= 300]
            #check if there are multiple values in the style of: 5-25%
            if len(re.findall(r"(\d+-\d+)ml", check, re.IGNORECASE)) > 0:
                volume_ml = []
            #we use the volume_ml we found if there is only one
            if (((len(volume_ml) == 1) | (all_equal(volume_ml) & (len(volume_ml) != 0)))):
                volume_ml = volume_ml[0]
                volume_exists = True
            else:
                volume_ml = description.replace(",",".")
                check = volume_ml.replace(".","")
                volume_ml = re.findall(r"(\d+\.\d+|\d+)ml", volume_ml, re.IGNORECASE)
                volume_ml = [float(l) for l in volume_ml]
                volume_ml = [x for x in volume_ml if x <= 300]
                #check if there are multiple values in the style of: 5-25%
                if len(re.findall(r"(\d+-\d+)ml", check, re.IGNORECASE)) > 0:
                    volume_ml = []
                #Check if there are different ml numbers, in which case we can not automatically determin the correct one.
                if (((len(volume_ml) == 1) | (all_equal(volume_ml) & (len(volume_ml) != 0)))):
                    volume_ml = volume_ml[0]
                    volume_exists = True
                else: 
                    volume_ml = []
                    volume_exists = False
        except Exception as e:
            print("get_volume_ml-single raises error:")
            print(e)
            volume_ml = []
            volume_exists = False
            
    return volume_ml, volume_exists

#### get_multiples

In [12]:
def get_multiples (name, description, x_times_only = False):
    multiples = []
    if x_times_only == False:
        if len(re.findall("two", name, re.IGNORECASE)) > 0:
            multiples.append(2.0)
        if len(re.findall("two", description, re.IGNORECASE)) > 0:
            multiples.append(2.0)
        if len(re.findall("double", name, re.IGNORECASE)) > 0:
            multiples.append(2.0)
        if len(re.findall("double", description, re.IGNORECASE)) > 0:
            multiples.append(2.0)
        if len(re.findall("three", name, re.IGNORECASE)) > 0:
            multiples.append(3.0)
        if len(re.findall("three", description, re.IGNORECASE)) > 0:
            multiples.append(3.0)
        if len(re.findall("triple", name, re.IGNORECASE)) > 0:
            multiples.append(3.0)
        if len(re.findall("triple", description, re.IGNORECASE)) > 0:
            multiples.append(3.0)

    check = re.findall(r"(\d+)x", name, re.IGNORECASE)
    if len(check) > 0:
        for k in range(len(check)):
            multiples.append(check[k])
    check = re.findall(r"(\d+)x", description, re.IGNORECASE)
    if len(check) > 0:
        for k in range(len(check)):
            multiples.append(check[k])
            
    multiples = [float(l) for l in multiples]
            
    return multiples
app.status = 2

#### get_summary_df

In [13]:
def get_summary_df (test_df):

    DE_df = test_df[test_df["Country"].str.contains("DE")]
    NL_df = test_df[test_df["Country"].str.contains("NL")]
    AUS_df = test_df[test_df["Country"].str.contains("AUS")]
    UK_df = test_df[test_df["Country"].str.contains("UK")]
    ES_df = test_df[test_df["Country"].str.contains("ES")]
    CH_df = test_df[test_df["Country"].str.contains("CH")]
    FR_df = test_df[test_df["Country"].str.contains("FR")]
    US_df = test_df[test_df["Country"].str.contains("USA")]
    EU_df = test_df[test_df["Country"].str.contains("EU|DE|NL|AUS|ES|CH|FR")]

    #append all the subsets to a list
    country_df_list = []
    country_df_list.append(test_df)
    country_df_list.append(DE_df)
    country_df_list.append(NL_df)
    country_df_list.append(AUS_df)
    country_df_list.append(UK_df)
    country_df_list.append(ES_df)
    country_df_list.append(CH_df)
    country_df_list.append(FR_df)
    country_df_list.append(US_df)
    country_df_list.append(EU_df)

    #make alist with the names of the countries
    country_list = ["All Countries","DE","NL","AUS","UK","ES","CH","FR","USA","EU"]


    #calculate the mean min and max
    mean_list = []
    max_list = []
    min_list = []
    number_products = []



    for i in range(len(country_df_list)):
        mean = np.mean(country_df_list[i]["€/mg"])
        max_ = np.max(country_df_list[i]["€/mg"])
        min_ = np.min(country_df_list[i]["€/mg"])

        max_ = round_to_2(max_)
        mean = round_to_2(mean)
        min_ = round_to_2(min_)

        max_list.append(max_)
        min_list.append(min_)
        mean_list.append(mean)
        number_products.append(len(country_df_list[i]))


    data = {"Country":country_list,
            "Average[€/mg]":mean_list, 
            "Minimum[€/mg]":min_list,
            "Maximum[€/mg]":max_list,
            "Number of Products":number_products}
    Summary_df = pd.DataFrame(data)

    return Summary_df

#### get_subset_df

In [14]:
def get_subset_df (temp_df, ml, percent):
    temp_df = temp_df.rename(columns={"CBD[mg]": "cbd"})
    temp_df = temp_df.rename(columns={"Volume[ml]": "ml"})
    temp_df = temp_df[temp_df.ml == ml]
    mg = percent * ml * 10
    temp_df = temp_df[temp_df.cbd == mg]
    temp_df = temp_df.rename(columns={"cbd": "CBD[mg]"})
    temp_df = temp_df.rename(columns={"ml": "Volume[ml]"})
    
    return temp_df

## Web Scraping Functions

In [15]:
Error_list =[]

### CBDVITAL 
Creating the Scraping Function for CBDVITAL

In [16]:
def scrape_CBDVITAL (URL_list):
    brand = "CBDVITAL" 
    country = "DE"
    website = "cbd-vital.de"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    for i in range(len(URL_list)):
        
        #driver = webdriver.Edge(options = options)
        driver = Edge(edge_path(), options = options)
        driver.get(URL_list[i])     
        
        for j in range(2): #iteration for the 2 Buttons (10ml and 30ml)
            
            
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME,'swatch-opt'))) #Somtimes the page doesn't load quickly enough, this is here to prevent that.        
            edge_soup = BeautifulSoup(driver.page_source,'lxml')        
            soup = BeautifulSoup(requests.get(URL_list[i], headers=headers).content,'lxml') #pull the website 
            
            #Get the Price    
            price = edge_soup.find("div", class_="product-info-price")
            price = price.find_all("span", class_="price")                           #find both the old and the current price
            price = price[1]                                                         #select only the current price
            price = price.text.strip()                                               #remove html code
            price = price.replace("€","") 
            price = price.replace(",",".")
            price = float(price) 
            price_list.append(price)

            #Get the name
            name = edge_soup.find("h1", class_="page-title")
            name = name.find("span", itemprop="name")
            name = name.text.strip()
            name_list.append(name)

            #Get the volume_ml
            volume_ml = edge_soup.find("div", class_="swatch-opt")        
            volume_ml = volume_ml.find("span", class_="swatch-attribute-selected-option")       
            volume_ml = volume_ml.text.strip()
            volume_ml = volume_ml.replace("ml","")
            volume_ml = float(volume_ml)
            volume_list.append(volume_ml)

            #Get mg_cbd        
            mg_cbd = edge_soup.find("span", class_="product-info-attribute-item product-config-cbd-gehalt")
            mg_cbd = mg_cbd.text.strip()
            mg_cbd = mg_cbd.replace(" mg CBD-Gehalt)","")
            mg_cbd = mg_cbd.replace("(","")
            mg_cbd = float(mg_cbd)
            cbd_list.append(mg_cbd)
            
            #add brand 
            brand_list.append(brand)
            country_list.append(country)
            website_list.append(website)
            
            driver.find_element_by_id("option-label-inhalt-138-item-13").click()
        
        driver.quit()
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}
    
    temp_df = pd.DataFrame(data)
    
    return temp_df
        
       
    

Selecting the Links from the URL_df which correspond to CBDVITAL and creating a list out of them 

In [17]:
#Select the Corresponding URLs from the URL_df     
URLs = URL_df.loc[URL_df["Company"] == "CBDVITAL"].loc[:,"Link"].tolist()

#Run the Scraper
try:
    Data_df = scrape_CBDVITAL(URLs)
except:
    try:
        Data_df = scrape_CBDVITAL(URLs)
    except:
        try:
            Data_df = scrape_CBDVITAL(URLs)
        except:
            Data_df = pd.DataFrame({"Country":["ERROR"], 
                                    "Website":["ERROR"], 
                                    "Company":["ERROR in CBDVITAL Scraper"],
                                    "Product":["ERROR in CBDVITAL Scraper"],
                                    "Price[€]":[np.nan],
                                    "Volume[ml]":[np.nan],
                                    "CBD[mg]":[np.nan]})
            Error_list.append("scrape_CBDVITAL")
num = 0 
num += 1
app.status = get_status(num)

### Nordic Oil

In [18]:
def scrape_Nordic_Oil (URL_list):
    brand = "Nordic Oil" 
    country = "DE/NL"
    website = "nordicoil.de"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    for i in range(len(URL_list)):
        
              
        soup = BeautifulSoup(requests.get(URL_list[i], headers=headers).content,'lxml') #pull the website 
            
        #Get the Price    
        price = soup.find("span", class_="prod-card__price text-xl font-bold")
        price = price.text.strip()                                               #remove html code
        price = price.replace("€","") 
        price = price.replace(",",".")
        price = float(price) 
        price_list.append(price)

        #Get the name
        name = soup.find("h1", class_="text-xxxl font-extrabold margin-bottom-xxxxs")
        name = name.text.strip()
        name_list.append(name)

        #Get the volume_ml
        volume_ml = soup.find("div", class_="prod-card__options margin-bottom-xs")        
        volume_ml_remove = volume_ml.find("span", class_="text-sm title-grey font-italic") #find the text that needs to be removed  
            
        volume_ml = volume_ml.text.strip()
        volume_ml_remove = volume_ml_remove.text.strip()
            
        volume_ml = volume_ml.replace(volume_ml_remove,"") #Remove the extra text
        volume_ml = volume_ml.replace("ml","")
        volume_ml = float(volume_ml)
        volume_list.append(volume_ml)

        #Get mg_cbd        
        mg_cbd = soup.find("ul", class_="list list-check margin-bottom-md")
        mg_cbd = mg_cbd.text.strip()
        mg_cbd = re.search(r"\(([A-Za-z0-9_]+)\)", mg_cbd) #search the string for text within brakets (The CBD content on the website is displayed in brackets)
        mg_cbd = mg_cbd.group(1) #Extract the first instance of brackets from the re object.
        mg_cbd = mg_cbd.replace("mg","")
        mg_cbd = float(mg_cbd)
        cbd_list.append(mg_cbd)
            
        #add brand 
        brand_list.append(brand)
        country_list.append(country)
        website_list.append(website)    
            
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}
    
    temp_df = pd.DataFrame(data)
    
    return temp_df
        
       
    

In [19]:
#Select the Corresponding URLs from the URL_df     
URLs = URL_df.loc[URL_df["Company"] == "Nordic Oil"].loc[:,"Link"].tolist() #make list from URls containing the company name "Nordic Oil"

#Run the Scraper
try:
    Data_df = pd.concat([Data_df, scrape_Nordic_Oil(URLs)]) #Call webscraper and merge results with already existing df
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Nordic_Oil(URLs)]) #Call webscraper and merge results with already existing df
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Nordic_Oil(URLs)]) #Call webscraper and merge results with already existing df
            Data_df.reset_index()
        except:
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],                 
                                                        "Company":["ERROR in Nordic Oil Scraper"],
                                                        "Product":["ERROR in Nordic Oil Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Nordic_Oil")
num += 1
app.status = get_status(num)

### Bol.com

In [20]:
def scrape_Bol (URL_list):
    country = "NL"
    website = "bol.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    stop = False
    
    ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)
    
    driver = Edge(edge_path(), options = options)
    driver.get(URL_list)   
        
    #Try if there is an accept cookies banner in the way, and click accept if that is the case
    try:

        driver.find_element_by_css_selector(".ui-btn[data-test='consent-modal-confirm-btn']") 
    except NoSuchElementException: #gets triggered if the cookie banner is not found
        a = 1
    else: #gets triggered if there is a cookie banner
        driver.find_element_by_css_selector(".ui-btn[data-test='consent-modal-confirm-btn']").click()

    #Continue for all pages of the search
    while stop == False:

        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,'product-item--row'))) #Somtimes the page doesn't load quickly enough, this is here to prevent that.        
        edge_soup = BeautifulSoup(driver.page_source,'lxml')        
        #soup = BeautifulSoup(requests.get(URL_list[i], headers=headers).content) #pull the website 

        #find all products on the page
        products_list = edge_soup.find_all("li", class_= "product-item--row")
        
        for i in range(len(products_list)):
            
            product = products_list[i]

            #Get the Price    
            price = product.find("div", class_="price-block__price")
            
            #sometimes the scraper finds objects that are hidden on the website, which don't have a price, etc.
            #If this is the case, the loop skips this product.
            if price is None:
                continue
                
            price = str(price)
            price = re.search('<meta content=(.*)itemprop', price)
            price = price.group(1)
            price = price.replace('"','')
            price = float(price) 

            #Get the brand
            brand = product.find("a", itemprop="name")
            brand = brand.text.strip()

            #Get the name
            name = product.find("a", class_="product-title") 
            name = name.text.strip()

            #Remove Spaces
            title_str = name.replace(" ","")
            title_str = title_str.replace(".","")

            #Get the volume_ml
            volume_ml = re.findall(r"(\d+)ml", title_str) #select all numbers before ml
            volume_ml = [float(j) for j in volume_ml]     #turn list of strings into list of floats
            volume_ml = sorted(volume_ml)                


            #Get mg_cbd        
            mg_cbd = re.findall(r"(\d+)mg", title_str) #select all numbers before mg
            mg_cbd = [float(k) for k in mg_cbd]        #turn list of strings into list of floats
            mg_cbd = sorted(mg_cbd)


            #Get % CBD
            percent_cbd = re.findall(r"(\d+)%", title_str) #select all numbers before %
            percent_cbd = [float(l) for l in percent_cbd]  #turn list of strings into list of floats
            percent_cbd = sorted(percent_cbd)
            #Check if the percent value (if one exists) is larger than 30%, and remove it if that is the case
            #This is to get rid of the "100% natural/vegan/biological" nonsense
            if len(percent_cbd) != 0:
                if percent_cbd[0] > 30:
                    percent_cbd = []

            #Find out if the title contains multiple products.
            multiples = re.findall(r"(\d+)x", title_str)
            multiples = [float(m) for m in multiples]
            multiples = sorted(multiples)
            #if there are multiples, set them for later division of endresult
            if len(multiples) != 0:
                multiples = multiples[0]
            else:
                multiples = 1

            #find out if the title contains + so the product number can not be accuratly determined
            #If there is a plus skip this step of the loop
            pluses = title_str.count("+")
            if pluses != 0:
                continue

            #Find out if the necessary information was in the title
            if len(mg_cbd) != 0:
                cbd_list.append(mg_cbd[0] * multiples)
                price_list.append(price)
                name_list.append(name)
                brand_list.append(brand)
                country_list.append(country)
                website_list.append(website)

                if len(volume_ml) != 0:
                    volume_list.append(volume_ml[0] * multiples)
                else:
                    volume_list.append(0)
            #mg cbd can be calculated if we have both the % and volume data 
            else: 
                if ((len(volume_ml) != 0) & (len(percent_cbd) != 0)):
                    mg_cbd = volume_ml[0] * percent_cbd[0] / 100 * 1000 * multiples
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml[0] * multiples)
                    price_list.append(price)
                    name_list.append(name)
                    brand_list.append(brand)
                    country_list.append(country)
                    website_list.append(website)

            # If we have neither cbd nor volume+% nothing is appended and the product is skipped

        try:    
            driver.find_element_by_css_selector(".ui-btn[aria-label='volgende']") 
        except NoSuchElementException: #gets triggered if the last page is reached
            stop = True
        else: #gets triggered if there are more search pages and clicks on the next button
            driver.refresh()

            WebDriverWait(driver, 20,ignored_exceptions=StaleElementReferenceException).until(EC.presence_of_element_located((By.CSS_SELECTOR,".ui-btn[aria-label='volgende']"))).click()



    driver.quit()
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [21]:
#Run the Scraper
#print (scrape_Bol("https://www.bol.com/nl/nl/s/?page=1&searchtext=CBD+olie&view=list&sort=price0&rating=all")) 
try:
    Data_df = pd.concat([Data_df, scrape_Bol("https://www.bol.com/nl/nl/s/?page=1&searchtext=CBD+olie&view=list&sort=price0&rating=all")]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Bol("https://www.bol.com/nl/nl/s/?page=1&searchtext=CBD+olie&view=list&sort=price0&rating=all")]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Bol("https://www.bol.com/nl/nl/s/?page=1&searchtext=CBD+olie&view=list&sort=price0&rating=all")]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Bol.com Scraper"],
                                                        "Product":["ERROR in Bol.com Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Bol")

num += 1
app.status = get_status(num)

### Cannabas 

In [22]:
def scrape_Cannabas (URL_list):
   
    brand = "Cannabas"
    country = "NL"
    website = "cannabas.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    
    ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)
    

    for i in range(len(URL_list)):

        driver = Edge(edge_path(), options = options)
        driver.get(URL_list[i]) 

        #Continue for all pages of the search
        j = 0
        while j < 2:

            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,'woocommerce-product-attributes-item'))) #Somtimes the page doesn't load quickly enough, this is here to prevent that.        
            edge_soup = BeautifulSoup(driver.page_source,'lxml')        
            #soup = BeautifulSoup(requests.get(URL_list[i], headers=headers).content) #pull the website 
            
            
            #Get the Price    
            price = edge_soup.find("p", class_="price")
            price = price.find_all("bdi")
            if len(price) == 0: #sometimes there is no <p for some reason
                price = edge_soup.find("span", class_="price")
                price = price.find_all("bdi")
            # if there is more than one price only the lower one is selected
            if len(price) == 1: #if there is only 1 price, no selection needs to occur
                price = price[0]
            else: #if there are two prices, the second one needs to be chosen on first interation of the loop and the first one on the second itteration
                if j == 0:
                    price = price[1]
                else:
                    price = price[0]

            price = price.text.strip()
            price = price.replace("€","")
            price = price.replace(",",".")
            price = float(price)
            
            #Get the name
            name = edge_soup.find("h1", class_="product_title entry-title")
            name = name.text.strip()

            #Remove Spaces
            title_str = name.replace(" ","")
            title_str = name.replace(",",".")

            #Get the volume_ml
            volume_ml = edge_soup.find("td", class_="woocommerce-product-attributes-item__value")
            volume_ml = volume_ml.text.strip()
            volume_ml = volume_ml.replace(" ","")
            volume_ml = re.findall(r"(\d+)ml", volume_ml) 
            volume_ml = [float(k) for k in volume_ml]
            volume_ml = sorted(volume_ml)
            #select between the two different puchase options (10ml and 30ml)
            if len(volume_ml) == 1: #if there are no two different volumes, there is no need to select
                volume_ml = volume_ml[0]
            else:
                if j == 0: #The First Itteration is the 30ml itteration 
                    volume_ml = volume_ml[1]
                else: # the second itteration is the 10ml itteration
                    volume_ml = volume_ml[0]


            #Get % CBD
            percent_cbd = re.search(r"(\d\.\d|\d+)%", title_str) #find digit-dot-digit or digit-digit before % in title-string
            percent_cbd = percent_cbd.group(1)
            percent_cbd = float(percent_cbd)

            #Get mg_cbd        
            mg_cbd = volume_ml * percent_cbd * 10 #/100 * 1000 = *10


            cbd_list.append(mg_cbd)
            volume_list.append(volume_ml)
            price_list.append(price)
            name_list.append(name)
            brand_list.append(brand)
            country_list.append(country)
            website_list.append(website)
            # If we have neither cbd nor volume+% nothing is appended and the product is skipped

            try:    
                driver.find_element_by_css_selector(".ivpa_term[data-term='10ml-en']") 
            except NoSuchElementException: #gets triggered if the last page is reached
                j = 2
            else: #gets triggered if there are more search pages and clicks on the next button
                driver.refresh()
                j = j + 1
                WebDriverWait(driver, 20,ignored_exceptions=StaleElementReferenceException).until(EC.presence_of_element_located((By.CSS_SELECTOR,".ivpa_term[data-term='10ml-en']"))).click()


        driver.quit()

    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [23]:
#Run the Scraper
URLs = URL_df.loc[URL_df["Company"] == "Cannabas"].loc[:,"Link"].tolist() #make list from URls containing the company name 
try:
    Data_df = pd.concat([Data_df, scrape_Cannabas(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Cannabas(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Cannabas(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Cannabas Scraper"],
                                                        "Product":["ERROR in Cannabas Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Cannabas")
num += 1
app.status = get_status(num)

### Swiss FX

In [24]:
def scrape_Swiss_FX (URL_list):
    brand = "Swiss FX" 
    country = "CH/DE/AUS"
    website = "swissfx.de"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    for i in range(len(URL_list)):
        
              
        soup = BeautifulSoup(requests.get(URL_list[i], headers=headers).content,'lxml') #pull the website 
            
        #Get the Price    
        price = soup.find("span", class_="sale-price")
        price = price.text.strip()                                               #remove html code
        price = price.replace("€","") 
        price = price.replace(",",".")
        price = float(price) 
        price_list.append(price)

        #Get the name
        name = soup.find("h1", class_="product-single__title")
        name = name.text.strip()
        name_list.append(name)
        
        #Get the product description containing volume and massCBD
        descr = soup.find("div", class_="product-single__description")
        descr = descr.find("li")
        descr = descr.text.strip()
        descr = descr.replace(" ","")
        
        #Get the volume_ml     
        volume_ml = re.findall(r"(\d+)ml", descr) 
        volume_ml = volume_ml[0]
        volume_ml = float(volume_ml)
        volume_list.append(volume_ml)

        #Get mg_cbd        
        mg_cbd = re.findall(r"(\d+)mg", descr) 
        mg_cbd = mg_cbd[0]
        mg_cbd = float(mg_cbd)
        cbd_list.append(mg_cbd)
            
        #add brand 
        brand_list.append(brand)
        country_list.append(country)
        website_list.append(website)    
            
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}
    
    temp_df = pd.DataFrame(data)
    
    return temp_df
        
       
    

In [25]:
#Run the Scraper
URLs = URL_df.loc[URL_df["Company"] == "Swiss FX"].loc[:,"Link"].tolist() #make list from URls containing the company name 
try:
    Data_df = pd.concat([Data_df, scrape_Swiss_FX(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Swiss_FX(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Swiss_FX(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in Swiss FX Scraper"],
                                                        "Product":["ERROR in Swiss FX Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Swiss_FX")
num += 1
app.status = get_status(num)

### reakiro

In [26]:
def scrape_reakiro (URL_list):
    brand = "reakiro" 
    country = "UK/EU"
    website = "cbdreakiro.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL_list, headers=headers).content,'lxml')
    
    #get all the price/info boxes
    products = soup.find_all("div", class_="p-2")
    
    for i in range(len(products)):
        
        product = products[i]
        #Get the Price    
        price = product.find("span", class_="oe_currency_value")
        price = price.text.strip()                                               #remove html code
        price = price.replace("€","") 
        price = price.replace(",",".")
        price = float(price) 
        price_list.append(price)

        #Get the name
        name = product.find("a", itemprop="name")
        name = name.text.strip()
        name_list.append(name)
       
        
        #Get the volume_ml     
       
        volume_list.append(np.nan)

        #Get mg_cbd 
        mg_cbd = name.replace(" ","")
        mg_cbd = re.findall(r"(\d+)mg", mg_cbd) 
        mg_cbd = mg_cbd[0]
        mg_cbd = float(mg_cbd)
        cbd_list.append(mg_cbd)
            
        #add brand 
        brand_list.append(brand)
        country_list.append(country)
        website_list.append(website)    
            
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}
    
    temp_df = pd.DataFrame(data)
    
    return temp_df

In [27]:
#Select the Corresponding URLs from the URL_df     
URLs = "https://cbdreakiro.com/shop/category/cbd-oils-11"

#Run the Scraper
try:
    Data_df = pd.concat([Data_df, scrape_reakiro(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_reakiro(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_reakiro(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in reakiro Scraper"],
                                                        "Product":["ERROR in reakiro Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_reakiro")
num += 1
app.status = get_status(num)

### CBDNOL

In [28]:
def scrape_CBDNOL (URL_list):
    brand = "CBDNOL" 
    country = "DE/AUS"
    website = "cbdnol.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    for i in range(len(URL_list)):
        
              
        soup = BeautifulSoup(requests.get(URL_list[i], headers=headers).content,'lxml') #pull the website 
            
        #Get the Price    
        price = soup.find("span", class_="product-price-minimum")
        price = price.text.strip()                                               #remove html code
        price = price.replace("€","") 
        price = price.replace(",",".")
        price = float(price) 
        price_list.append(price)

        #Get the name
        name = soup.find("div", class_="page-title")
        name = name.text.strip()
        name = name.replace("\n  \nCBDNOL","")
        name_list.append(name)
        
        #Get the Percentage CBD from the title
        percent = name.replace(" ","")
        percent = re.findall(r"(\d+)%", percent) 
        percent = percent[0]
        percent = float(percent)       
        
        #Get the volume_ml     
        volume_ml = soup.find("select", class_="single-option-selector")
        volume_ml = volume_ml.find("option", selected="selected")
        volume_ml = volume_ml.text.strip()
        volume_ml = volume_ml.replace(" ","")
        volume_ml = volume_ml.replace("ml","")        
        volume_ml = float(volume_ml)
        volume_list.append(volume_ml)

        #Get mg_cbd        
        mg_cbd = percent * volume_ml * 10
        cbd_list.append(mg_cbd)
            
        #add brand 
        brand_list.append(brand)
        
        country_list.append(country)
        website_list.append(website)    
            
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}
    
    temp_df = pd.DataFrame(data)
    
    return temp_df

In [29]:
#Run the Scraper
URLs = URL_df.loc[URL_df["Company"] == "CBDNOL"].loc[:,"Link"].tolist() #make list from URls containing the company name 
try:
    Data_df = pd.concat([Data_df, scrape_CBDNOL(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_CBDNOL(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_CBDNOL(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in CBDNOL Scraper"],
                                                        "Product":["ERROR in CBDNOL Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_CBDNOL")
num += 1
app.status = get_status(num)

### Hanfama

In [30]:
def scrape_Hanfama (URL):
    brand = "Hanfama" 
    country = "DE/AUS"
    website = "hanfama.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("div", class_="product-small")
    

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
        
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        #print(link_list[i])

              
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml') #pull the website 
        
        #Get Price(s) and pick lowest one
        price = soup.find("p", class_="product-page-price").text.strip()
        price = price.replace(" ","")
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+)€", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0] 

        #Get the name
        name = soup.find("h1", class_="product_title")
        name = name.text.strip()
        
        #Get the Product description
        description_part = soup.find("div", class_="product-short-description").text.strip()
        description = description_part + soup.find("div", class_="tab-panels").text.strip()
        description = description.replace(" ","")
        description_part = description_part.replace(" ","")
        
        #Find out if the title contains multiple products.
        multiples = get_multiples(name, description_part, x_times_only = True)

         #if there are multiples, set them for later division of endresult
        #print(multiples)
        if len(multiples) == 0:
            multiples = 1
        else:
            if all_equal(multiples):
                multiples = multiples[0]
            else:
                #print(name)
                #print("skip because multiples were not all equal")
                continue
            
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]
            
        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       
         
        
        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    
        
        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]   
        #print(name)
        #print("MG: " + str(mg_cbd_exists) + "   ML: " + str(volume_exists) + "   %: " + str(percent_exists)) 
        #print("multiples: " +str(multiples))
        #print(mg_cbd * multiples)
        #now to append the lists
        if mg_cbd_exists == True:
            #print("viable")
            mg_cbd = mg_cbd * multiples
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10 * multiples
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)
            
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}
    
    temp_df = pd.DataFrame(data)
    
    return temp_df

In [31]:
#Run the Scraper
URLs = "https://hanfama.com/cbd-oel/"
try:
    Data_df = pd.concat([Data_df, scrape_Hanfama(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Hanfama(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Hanfama(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Hanfama Scraper"],
                                                        "Product":["ERROR in Hanfama Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Hanfama")
num += 1
app.status = get_status(num)

### Hempcare

In [32]:
def scrape_Hempcare (URL):
    
    brand = "Hempcare"
    country = "CH"
    website = "cbd-hempcare.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("a", class_="product-name")

    link_list = []
    for i in range(len(link_basket)):
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = "https://www.cbd-hempcare.com" + temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
                
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        text = str(soup) 

        #Cut out a string snipet
        start = text.find("var meta") + len("var meta") 
        end = text.find("var attr in meta")
        substring = text[start:end]
        
        matches_volume = re.findall(r"public_title(.*?)sku", substring)
        for j in range(len(matches_volume)):
            matches_volume[j] = matches_volume[j].replace('"','')
            matches_volume[j] = matches_volume[j].replace(',','')
            matches_volume[j] = matches_volume[j].replace(':','')
            matches_volume[j] = matches_volume[j].replace('ml','')
            #for some products the volume is not in the metadata
            #in this case the volume is instead pulled from the text box
            try:
                matches_volume[j] = float(matches_volume[j])
                volume = [1]
            except:
                volume = soup.find("div", class_="rte")
                volume = volume.text.strip()
                volume = re.findall(r"(\d+)ml", volume)                
                matches_volume[j] = volume[0]
                matches_volume[j] = matches_volume[j].replace('ml','')
                matches_volume[j] = float(matches_volume[j])

        #find all matches for the price    
        matches_price = re.findall(r"price(.*?)name", substring)
        for j in range(len(matches_price)):
            matches_price[j] = matches_price[j].replace('"','')
            matches_price[j] = matches_price[j].replace(',','')
            matches_price[j] = matches_price[j].replace(':','')
            matches_price[j] = float(matches_price[j])
            matches_price[j] = matches_price[j]/100
        
        #If there was a reading error, we skip this product
        if len(matches_price) != len(matches_volume):
            continue
        #if we got multiple volume sizes the correct one can not be determined
        if len(volume) > 1:
            continue       
        
        #Get the name
        name = soup.find("div", class_="product-info") 
        name = name.find("h1")
        name = name.text.strip()

        #Remove Spaces
        title_str = name.replace(" ","")
        title_str = title_str.replace(",",".")

        #Get % CBD
        percent_cbd = re.findall(r"(\d+)%", title_str) #select all numbers before %
        percent_cbd = [float(l) for l in percent_cbd]  #turn list of strings into list of floats
        percent_cbd = sorted(percent_cbd)
        percent_cbd = percent_cbd[0]
        
        #Add all the values to the list
        for j in range(len(matches_price)):
            brand_list.append(brand)
            price_list.append(matches_price[j])
            name_list.append(name)
            volume_list.append(matches_volume[j])
            cbd_list.append(matches_volume[j] * percent_cbd * 10)
            country_list.append(country)
            website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}


    temp_df = pd.DataFrame(data)

    return temp_df
        

In [33]:
#Run the Scraper
URLs = "https://www.cbd-hempcare.com/collections/cbd-oel"
try:
    Data_df = pd.concat([Data_df, scrape_Hempcare(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Hempcare(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Hempcare(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in Hempcare Scraper"],
                                                        "Product":["ERROR in Hempcare Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Hempcare")
num += 1
app.status = get_status(num)

### Royal Queen Seeds

In [34]:
def scrape_Royal_Queen_Seeds (URL):
    
    brand = "Royal Queen Seeds"
    country = "INT/EU"
    website = "royalqueenseeds.de"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    product_list = soup.find_all("li", class_="ajax_block_product")

    #go through all CBD Oil Products
    for i in range(len(product_list)):
                
        product = product_list[i]
        
        #Get the Price
        price = product.find("div", class_="prices-block")
        #There are two prices, "price" and "discount hide" the lower one will be selected.
        price_1 = price.find("span", class_="discount")
        price_1 = price_1.text.strip()
        price_1 = price_1.replace(",",".")
        price_1 = price_1.replace(" ","")
        price_1 = price_1.replace("€","")
        price_1 = float(price_1)
        price_2 = price.find("span", class_="price")
        price_2 = price_2.text.strip()
        price_2 = price_2.replace(",",".")
        price_2 = price_2.replace(" ","")
        price_2 = price_2.replace("€","")
        price_2 = float(price_2)
        # now select the lower price
        two_prices = [price_1,price_2]
        two_prices = sorted(two_prices)
        price = two_prices[0]
        
        #Get the name
        name = product.find("div", class_="product-title") 
        name = name.find("a", class_="product-link")
        name = name.text.strip()

        #Remove Spaces
        title_str = name.replace(" ","")
        title_str = title_str.replace(",",".")

        #Get % CBD
        percent_cbd = re.findall(r"(\d\.\d|\d+)%", title_str) #select all numbers before %
        #if there is more than 1 element, the oil is not pure cbd and will be skipped
        if len(percent_cbd) > 1:
            continue
        percent_cbd = [float(l) for l in percent_cbd]  #turn list of strings into list of floats
        percent_cbd = sorted(percent_cbd)
        percent_cbd = percent_cbd[0]
        
        #get the volume
        volume_ml = product.find("a", class_="product-combination")
        volume_ml = volume_ml.text.strip()        
        volume_ml = volume_ml.replace(" ","")
        volume_ml = re.search(r"(\d+)ml", volume_ml).group(1)
        volume_ml = float(volume_ml)
        
        #Get the mg CBD
        mg_cbd = re.findall(r"(\d+)mg", title_str)
        #if there is more than 1 element, the oil is not pure cbd and will be skipped
        if len(mg_cbd) > 1:
            continue
        if len(mg_cbd) == 0:
            mg_cbd = volume_ml * percent_cbd * 10
        else:
            mg_cbd = float(mg_cbd[0])
        
        #Add all the values to the list
        brand_list.append(brand)
        price_list.append(price)
        name_list.append(name)
        volume_list.append(volume_ml)
        cbd_list.append(mg_cbd)
        country_list.append(country)
        website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df        

In [35]:
#Run the Scraper
URLs = "https://www.royalqueenseeds.de/46-cbd-oel"
try:
    Data_df = pd.concat([Data_df, scrape_Royal_Queen_Seeds(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Royal_Queen_Seeds(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Royal_Queen_Seeds(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Royal Queen Seeds Scraper"],
                                                        "Product":["ERROR in Royal Queen Seeds Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Royal_Queen_Seeds")
            
num += 1
app.status = get_status(num)

### CBD Pharma NaturalCare

In [36]:
def scrape_CBD_Pharma_NaturalCare (URL):
    
    brand = "CBD Pharma NaturalCare"
    country = "ES/CH"
    website = "cbdpharmanaturalcare.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    product_list = soup.find_all("div", class_="cbd-dosis")
    
    #go through all CBD Oil Products
    for i in range(len(product_list)):
                
        product = product_list[i]
        
        #Get the Price
        price = product.find("div", class_="cbd-price")
        #There are two prices, "price" and "discount hide" the lower one will be selected.
        price = price.find("bdi")
        price = price.text.strip()
        price = price.replace(",",".")
        price = price.replace(" ","")
        price = price.replace("€","")
        price = float(price)
        
        #Get % CBD
        percent_cbd = product.find("h3")
        percent_cbd = percent_cbd.text.strip()
        percent_cbd = percent_cbd.replace(",",".")
        percent_cbd = percent_cbd.replace(" ","")
        percent_cbd = re.search(r"(\d\.\d|\d+)%", percent_cbd).group(1) #select all numbers before %
        percent_cbd = float(percent_cbd)
                
        #get the volume
        volume_ml = product.find("h3")
        volume_ml = volume_ml.find("span")
        volume_ml = volume_ml.text.strip()        
        volume_ml = volume_ml.replace(" ","")
        volume_ml = re.search(r"(\d+)ml", volume_ml).group(1)
        volume_ml = float(volume_ml)
        
        #Get the mg CBD
        mg_cbd = product.find("div", class_="cbd-qty")
        mg_cbd = mg_cbd.text.strip()
        mg_cbd = mg_cbd.replace(" ","")
        mg_cbd = re.search(r"(\d+)mg", mg_cbd).group(1)
        mg_cbd = float(mg_cbd)
                    
        #Get the name
        name = "Serum de CBD - " + str(percent_cbd) +"% - " + str(volume_ml) + "ml - (" + str(mg_cbd) + "mg)"
        
        #Add all the values to the list
        brand_list.append(brand)
        price_list.append(price)
        name_list.append(name)
        volume_list.append(volume_ml)
        cbd_list.append(mg_cbd)
        country_list.append(country)
        website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df        

In [37]:
#Run the Scraper
URLs = "https://cbdpharmanaturalcare.com/serum-cbd/"
try:
    Data_df = pd.concat([Data_df, scrape_CBD_Pharma_NaturalCare(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_CBD_Pharma_NaturalCare(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_CBD_Pharma_NaturalCare(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in CBD Pharma NaturalCare Scraper"],
                                                        "Product":["ERROR in CBD Pharma NaturalCare Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_CBD_Pharma_NaturalCare")
            
num += 1
app.status = get_status(num)

### Dutch Headshop

In [38]:
def scrape_Dutch_Headshop (URL):
    
    country = "NL/EU"
    website = "dutch-headshop.de"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("li", class_="item")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False
        brand = "Dutch Headshop"        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("span", class_="h1")
        name = name.text.strip()
        
        #if the name does not contain "Öl" we skip this product
        if (len(re.findall("Öl", name, re.IGNORECASE)) == 0):
            #print("No öl")
            continue
        
        #Check for populare Brands
        if (len(re.findall("Jacob Hooy", name, re.IGNORECASE)) != 0):
            brand = "Jacob Hooy"
        if (len(re.findall("Hemptouch", name, re.IGNORECASE)) != 0):
            brand = "Hemptouch"
        if (len(re.findall("Hemplife", name, re.IGNORECASE)) != 0):
            brand = "Hemplife"
        if (len(re.findall("Medihemp", name, re.IGNORECASE)) != 0):
            brand = "Medihemp"
        
        #Get the Product description
        description = soup.find("div", class_="short-description").text.strip()
        description = description + soup.find("div", class_="full-description").text.strip()
        description = description.replace(" ","")
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]  

        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue            
        
        
        #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
        price = soup.find("ul", class_="product-list")
        if price is None:
            price = soup.find("div", class_="price-info")
            price = price.find_all("span", class_="price")
            # In case there is discount, both prices are recorded and the lower one is picked
            for j in range(len(price)):
                price[j] = price[j].text.strip()
                price[j] = price[j].replace('€','')
                price[j] = price[j].replace(',','.')
                price[j] = price[j].replace(' ','')
                price[j] = float(price[j])
            price = sorted(price)
            price = price[0]
            info_list =[]
        else:
            info_list = price.find_all("div", class_="info")
            price_s = []
            volume_s = []
            #We can also get the volume from the info list (if it exists)
            for j in range(len(info_list)):
                price = info_list[j].find_all("span", class_="price")
                volume_ml = info_list[j].find("h3", class_="name").text.strip()
                volume_ml = volume_ml.replace("ml","")
                volume_ml = volume_ml.replace(" ","")
                volume_ml = float(volume_ml)
                
                # In case there is discount, both prices are recorded and the lower one is picked
                for k in range(len(price)):
                    price[k] = price[k].text.strip()
                    price[k] = price[k].replace('€','')
                    price[k] = price[k].replace(',','.')
                    price[k] = price[k].replace(' ','')
                    price[k] = float(price[k])
                price = sorted(price)
                price = price[0]
                #append the multiple prices and volumes to their respective lists.
                price_s.append(price)
                volume_s.append(volume_ml)
                volume_exists = True
        
        #search vor volume if no volume exists yet
        if volume_exists == False:
            get = get_volume_ml(name, description)
            volume_ml = get[0]
            volume_exists = get[1] 

        #now to append the lists
        if len(info_list) == 0: #This if checks if multiple versions of the same product with different ml were on the page. yes -> else, no -> if
            #print("single product")
            #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
            if mg_cbd_exists == True:
                #print("viable")
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                country_list.append(country)
                website_list.append(website)
                if volume_exists == True:
                    volume_list.append(volume_ml)
                else:
                    volume_list.append(np.nan)
            else:
                if ((volume_exists == True) & (percent_exists == True)):
                    #print("viable")
                    mg_cbd = volume_ml * percent_cbd * 10
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)
        else:
            #print("multiple products")
            #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists))
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                for m in range(len(info_list)):
                    brand_list.append(brand)
                    name_list.append(name)
                    mg_cbd = volume_s[m] * percent_cbd * 10
                    cbd_list.append(mg_cbd)
                    price_list.append(price_s[m])
                    volume_list.append(volume_s[m])
                    country_list.append(country)
                    website_list.append(website)
            else:
                continue
                
                
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [39]:
#Run the Scraper
URLs = "https://www.dutch-headshop.de/cbdshop?limit=all"
try:
    Data_df = pd.concat([Data_df, scrape_Dutch_Headshop(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Dutch_Headshop(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Dutch_Headshop(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Dutch Headshop Scraper"],
                                                        "Product":["ERROR in Dutch Headshop Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Dutch_Headshop")
num += 1
app.status = get_status(num)

### GreenBee-CBD.fr

In [40]:
def scrape_GreenBee_CBD (URL):
    
    brand = "GreenBee"
    country = "FR"
    website = "greenbee-cbd.fr"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("div", class_="frame_produit")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = "https://www.greenbee-cbd.fr" + temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False       
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("span", class_="hikashop_product_name_main")
        name = name.text.strip()
                       
        #Get the Product description
        description = soup.find("div", class_="hikashop_product_right_part").text.strip()
        description = description.replace(" ","")
        
        #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
        price = soup.find("div", class_="mobile_pricing")
        price = price.find("span", class_="hikashop_product_price_full")
        price = price.text.strip()
        price = price.replace(',','.')
        price = price.replace(' ','')
        price = re.findall(r"(\d+\.\d+|\d+)€", price)
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0]
        
              
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       


        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    

        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1] 
                
        #now to append the lists

        #print("single product")
        #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)
                      
                
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [41]:
#Run the Scraper
URLs = "https://www.greenbee-cbd.fr/produits-cbd/huiles.html"
try:
    Data_df = pd.concat([Data_df, scrape_GreenBee_CBD(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_GreenBee_CBD(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_GreenBee_CBD(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in GreenBee-CBD.fr Scraper"],
                                                        "Product":["ERROR in GreenBee-CBD.fr Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_GreenBee_CBD")
            
num += 1
app.status = get_status(num)

### Hexagon Verte

In [42]:
def scrape_Hexagon_Vert (URL):
    
    brand = "Hexagon Vert shop"
    country = "FR"
    website = "hexagonevert.fr"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find("div", class_="products")
    link_basket = link_basket.find_all("div", class_="box")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False
        brand = "Hexagon Vert" 
        
        try:
            soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')


            #Get the Name
            name = soup.find("h1", class_="product_title")
            name = name.text.strip()


            #Get Brand
            brand = soup.find("div", class_="pwb-single-product-brands").text.strip()

            #Get the Product description
            description = soup.find("div", id="tab-Informations")
            if description is None:
                description = soup.find("div", id="tab-informations")
            description = description.find("li").text.strip()
            description = description.replace(" ","")
            #check if cbg is mentioned in the description and exclude the product if that is the case
            if skip_if_no_oil(name, description):
                #print("no oil = skip")
                continue
            #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
            info_list = soup.find("div", class_="price-wrapper")
            info_list = info_list.find_all("span", class_="woocommerce-Price-amount")
            if len(info_list) == 1:
                price = info_list[0]
                price = price.text.strip()
                price = price.replace(" ","")
                price = price.replace(",",".")
                price = re.findall(r"(\d+\.\d+|\d+)€", price)
                price = [float(l) for l in price]
                price = sorted(price)
                price = price[0]

            if len(info_list) == 2:

                price_s = []
                #We can also get the volume from the info list (if it exists)
                for j in range(len(info_list)):
                    price = info_list[j].text.strip()
                    price = price.replace(" ","")
                    price = price.replace(",",".")
                    price = re.findall(r"(\d+\.\d+|\d+)€", price)
                    price = [float(l) for l in price]
                    price = sorted(price)
                    price = price[0]                

                    #append the multiple prices 
                    price_s.append(price)
                price_s = sorted(price_s)
            #Too many or no prices were found
            if (len(info_list) == 0) | (len(info_list) > 2):
                continue

            #Get the percentage of CBD
            get = get_percent_cbd(name, description)
            percent_cbd = get[0]
            percent_exists = get[1]

            #Get the mg_cbd
            try:
                #if there is only one price
                if len(info_list) == 1:
                    get = get_mg_cbd(name, description)
                    mg_cbd = get[0]
                    mg_cbd_exists = get[1]    
                else:
                    mg_cbd = description.replace(".","")
                    mg_cbd = re.findall(r"(\d\.\d|\d+)mg", mg_cbd)
                    mg_cbd = [float(l) for l in mg_cbd]
                    mg_cbd = sorted(mg_cbd)
                    #Check if there are different % numbers, in which case we can not automatically determin the correct one.
                    if all_equal(mg_cbd):
                        continue
                    else: 
                        mg_cbd_s = [mg_cbd[0], mg_cbd[1]]
                        mg_cbd_exists = True
            except:
                mg_cbd_exists = False

            # If we have neither a percentage nor a mg value, we can not use the data
            if (mg_cbd_exists == False) & (percent_exists == False):
                #print("mg false and percent false")
                continue            


             #Get the volume_ml
            try:
                #if there is only one price
                if len(info_list) == 1:
                    get = get_volume_ml(name, description)
                    volume_ml = get[0]
                    volume_exists = get[1] 
                else:
                    volume_ml = description.replace(",",".")
                    volume_ml = re.findall(r"(\d+\.\d+|\d+)ml", volume_ml)
                    volume_ml = [float(l) for l in volume_ml]
                    volume_ml = sorted(volume_ml)
                    #Check if there are different % numbers, in which case we can not automatically determin the correct one.
                    if all_equal(volume_ml):
                        continue
                    else: 
                        volume_s = [volume_ml[0], volume_ml[1]]
                        volume_exists = True
            except:
                volume_exists = False        

            #now to append the lists
            if len(info_list) == 1: #This if checks if multiple versions of the same product with different ml were on the page. yes -> else, no -> if
                #print("single product")
                #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
                if mg_cbd_exists == True:
                    #print("viable")
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    country_list.append(country)
                    website_list.append(website)
                    if volume_exists == True:
                        volume_list.append(volume_ml)
                    else:
                        volume_list.append(np.nan)
                else:
                    if ((volume_exists == True) & (percent_exists == True)):
                        #print("viable")
                        mg_cbd = volume_ml * percent_cbd * 10
                        brand_list.append(brand)
                        price_list.append(price)
                        name_list.append(name)
                        cbd_list.append(mg_cbd)
                        volume_list.append(volume_ml)
                        country_list.append(country)
                        website_list.append(website)
            else:
                #print("multiple products")
                #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists))
                if mg_cbd_exists == True:
                    for m in range(len(info_list)):
                        brand_list.append(brand)
                        name_list.append(name)
                        cbd_list.append(mg_cbd_s[m])
                        price_list.append(price_s[m])
                        country_list.append(country)
                        website_list.append(website)
                        if volume_exists == True:
                            volume_list.append(volume_s[m])
                        else:
                            volume_list.append(np.nan)

                else:
                    if ((volume_exists == True) & (percent_exists == True)):
                    #print("viable")
                        for m in range(len(info_list)):
                            brand_list.append(brand)
                            name_list.append(name)
                            mg_cbd = volume_s[m] * percent_cbd * 10
                            cbd_list.append(mg_cbd)
                            price_list.append(price_s[m])
                            volume_list.append(volume_s[m])
                            country_list.append(country)
                            website_list.append(website)
                    else:
                        continue

        except Exception as e:            
            print("----------------------------------------")            
            print("link:" + str(i+1) + "  of:" + str(len(link_list)))
            print("Error raised:")
            print(e)
            
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [43]:
#Run the Scraper
URLs = "https://www.hexagonevert.fr/categorie-produit/huile-cbd/"
try:
    Data_df = pd.concat([Data_df, scrape_Hexagon_Vert(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Hexagon_Vert(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Hexagon_Vert(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in Hexagon Verte.fr Scraper"],
                                                        "Product":["ERROR in Hexagon Verte.fr Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Hexagon_Vert")
            
num += 1
app.status = get_status(num)

### CBD Sense (= CBD Olie.nl)

In [44]:
def scrape_CBD_Sense (URLs):
    brand = "CBD_Sense(CBD_Olie)"
    country = "DE/NL/EU"
    website = "cbdsense.de = cbdolie.nl"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
       
    #go through 4 settings: 10ml,30ml,100ml, 250ml
    for i in range(len(URLs)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        soup = BeautifulSoup(requests.get(URLs[i], headers=headers).content,'lxml')
        
        #select the correct volume_ml from the link
        volume_ml = float(re.findall(r"(\d+)ml", URLs[i])[0])
        
        product_list = soup.find_all("li", class_="product")
        
        for j in range(len(product_list)):
            
            product = product_list[j]    
            
            percent_exists = False
            mg_cbd_exists = False
            brand = "CBD Sense.de(CBD Olie.nl)"        



            #Get the Name
            name = product.find("span", class_="product-title")
            name = name.text.strip()


            #Check for populare Brands
            if (len(re.findall("Jacob Hooy", name, re.IGNORECASE)) != 0):
                brand = "Jacob Hooy"
            if (len(re.findall("Renova", name, re.IGNORECASE)) != 0):
                brand = "Renova"
            if (len(re.findall("Endoca", name, re.IGNORECASE)) != 0):
                brand = "Endoca"
            if (len(re.findall("Medihemp", name, re.IGNORECASE)) != 0):
                brand = "Medihemp"
            if (len(re.findall("Cibdol", name, re.IGNORECASE)) != 0):
                brand = "Cibdol"

            #Get the Product description
            description = name.replace(" ","")
            
            #check if cbg is mentioned in the description and exclude the product if that is the case
            if len(re.findall("cbg", name, re.IGNORECASE)) > 0:
                continue
           
            #Get Price(s) and pcik the lowest one
            price = product.find("span", class_="woocommerce-Price-amount")
            price = price.find_all("span", class_="amount")
            
            for k in range(len(price)):
                price[k] = price[k].text.strip()
                price[k] = price[k].replace(",",".")
                price[k] = price[k].replace("€","")
                price[k] = price[k].replace(" ","")
                price[k] = float(price[k])
            
            price = sorted(price)
            price = price[0]

            #Get percentage_cbd
            get = get_percent_cbd(name, description)
            percent_cbd = get[0]
            percent_exists = get[1]

            #Get the mg_cbd
            get = get_mg_cbd(name, description)
            mg_cbd = get[0]
            mg_cbd_exists = get[1]       

            # If we have neither a percentage nor a mg value, we can not use the data
            if (mg_cbd_exists == False) & (percent_exists == False):
                #print("mg false and percent false")
                continue            
                

            #now to append the lists
            if mg_cbd_exists == True:
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)
                
            else:
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    mg_cbd = volume_ml * percent_cbd * 10
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)
                

                
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [45]:
#Run the Scraper
URLs = URL_df.loc[URL_df["Company"] == "CBD Sense"].loc[:,"Link"].tolist()
try:
    Data_df = pd.concat([Data_df, scrape_CBD_Sense(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_CBD_Sense(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_CBD_Sense(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in CBD Sense Scraper"],
                                                        "Product":["ERROR in CBD Sense Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_CBD_Sense")
            
num += 1
app.status = get_status(num)

### CBD Brothers

In [46]:
def scrape_CBD_Brothers (URL):
    
    brand = "CBD Brothers"
    country = "DE"
    website = "cbd-brothers.de"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find("div", class_="products")
    link_basket = link_basket.find_all("article", class_="product-miniature")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("h1", class_="h1")
        name = name.text.strip()
        
        
        
        #Get the Product description
        description = soup.find("div", id="description").text.strip()
        description = description.replace(" ","")
        
        #check if cbg, wax or kristall is mentioned in the description and exclude the product if that is the case
        if len(re.findall("cbg", name, re.IGNORECASE)) > 0:
            continue
        if len(re.findall("Kristall", name, re.IGNORECASE)) > 0:
            continue
        if len(re.findall("Wax", name, re.IGNORECASE)) > 0:
            continue
        if len(re.findall("cbg", description, re.IGNORECASE)) > 0:
            continue
        #Check if Öl is mentioned and exclude it if not
        if (len(re.findall("Öl", name, re.IGNORECASE)) == 0):
            #print("No öl")
            continue
            
        #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
        price = soup.find("div", class_="product-prices").text.strip()
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+)€", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0]        
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       


        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    

        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]   

        #now to append the lists
        #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

                
                
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [47]:
#Run the Scraper
URLs = "https://www.cbd-brothers.de/7-cbd-dab-wax-cbg-cbd-oel"
try:
    Data_df = pd.concat([Data_df, scrape_CBD_Brothers(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_CBD_Brothers(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_CBD_Brothers(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in CBD_Brothers Scraper"],
                                                        "Product":["ERROR in CBD_Brothers Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_CBD_Brothers")
            
num += 1
app.status = get_status(num)

### Holistic Hemp Scotland

In [48]:
def scrape_Holistic_Hemp_Scotland (URL):
    
    brand = "Holistic Hemp Scotland"
    country = "UK"
    website = "holistichempscotland.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find("section", class_="products")
    link_basket = link_basket.find_all("article", class_="item")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("h2", class_="page-title")
        name = name.text.strip()
        
        
        
        #Get the Product description
        description = soup.find("div", id="ingredients").text.strip()
        description = description.replace('\n','')
        description = description.replace(" ","")
        
        #check if cbg, wax or kristall is mentioned in the description and exclude the product if that is the case
        if len(re.findall("cbg", name, re.IGNORECASE)) > 0:
            continue
        if len(re.findall("paste", name, re.IGNORECASE)) > 0:
            continue
        if len(re.findall("capsule", name, re.IGNORECASE)) > 0:
            continue
        if len(re.findall("cbg", description, re.IGNORECASE)) > 0:
            continue
        #Check if Öl is mentioned and exclude it if not
        if (len(re.findall("oil", name, re.IGNORECASE)) == 0):
            #print("No öl")
            continue
            
        #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
        info_list = soup.find("p", class_="price")
        info_list = info_list.find_all("span", class_="woocommerce-Price-amount")
        if len(info_list) == 1:
            price = info_list[0]
            price = price.text.strip()
            price = price.replace(" ","")
            price = price.replace(",",".")
            price = price.replace(".",".")
            price = price.replace("£","")
            price = str(price)
            price = float(price)
        #if there is more than one price we need to extract our data from the metadata    
        else:
            text = str(soup)
            text = text.replace(" ","")
            meta_data = re.search(r"variations_formcart(.*?)method",text).group(1)
            price_s = re.findall(r"(\d+\.\d+|\d+)&lt", meta_data)
            price_s = [float(l) for l in price_s]            
            price_s = list(set(price_s)) #remove duplicates
            price_s = sorted(price_s)
            
        #Get the name(s)
        if len(info_list) == 1:
            name = soup.find("h2", class_="page-title")
            name = name.text.strip()
        else:
            name_s = re.findall(r"description(.*?)\.&lt", meta_data)
            name_s = list(set(name_s))
            for j in range(len(name_s)):
                name_s[j] = name_s[j].replace("&quot;:&quot;&lt;p&gt;","")
        
        #Get the percentage(s) of CBD
        if len(info_list) == 1:
            get = get_percent_cbd(name, description)
            percent_cbd = get[0]
            percent_exists = get[1]
        else:
            get = get_percent_cbd("empty", name_s, multiple_options = True, option_s = name_s)
            percent_s = get[0]
            percent_exists = get[1]
            
            
        #Get the mg_cbd
        if len(info_list) == 1:
            get = get_mg_cbd(name, description)
            mg_cbd = get[0]
            mg_cbd_exists = get[1]    
        else:
            get = get_mg_cbd("empty", name_s, multiple_options = True, option_s = name_s)
            mg_cbd_s = get[0]
            mg_cbd_exists = get[1] 
            
            
        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue
        
         #Get the volume_ml
        if len(info_list) == 1:
            get = get_volume_ml(name, description)
            volume_ml = get[0]
            volume_exists = get[1] 
  
        else:
            get = get_volume_ml("empty", name_s, multiple_options = True, option_s = name_s)
            volume_ml = get[0]
            volume_exists = get[1] 
            

        #now to append the lists
        #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
        if len(info_list) == 1:
            if mg_cbd_exists == True:
                #print("viable")
                brand_list.append(brand)
                price_list.append(converter("GBP", price))
                name_list.append(name)
                cbd_list.append(mg_cbd)
                country_list.append(country)
                website_list.append(website)
                if volume_exists == True:
                    volume_list.append(volume_ml)
                else:
                    volume_list.append(np.nan)
            else:
                if ((volume_exists == True) & (percent_exists == True)):
                    #print("viable")
                    mg_cbd = volume_ml * percent_cbd * 10
                    brand_list.append(brand)
                    price_list.append(converter("GBP", price))
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)
        else:
            if (mg_cbd_exists == True) & (len(price_s) == len(mg_cbd_s) == len(name_s)):
                for j in range(len(price_s)):
                    brand_list.append(brand)
                    price_list.append(converter("GBP", price_s[j]))
                    name_list.append(name_s[j])
                    cbd_list.append(mg_cbd_s[j])
                    country_list.append(country)
                    website_list.append(website)
                    if volume_exists == True:
                        volume_list.append(volume_ml[j])
                    else:
                        volume_list.append(np.nan)
               
            else:
                if ((volume_exists == True) & (percent_exists == True)) & (len(price_s) == len(percent_s) == len(name_s)):
                    for j in range(len(price_s)):
                        brand_list.append(brand)
                        price_list.append(converter("GBP", price_s[j]))
                        name_list.append(name_s[j])
                        mg_cbd = percent_s[j] * volume_ml * 10
                        cbd_list.append(mg_cbd)
                        volume_list.append(volume_ml[j])
                        country_list.append(country)
                        website_list.append(website)
                
                
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [49]:
#Run the Scraper
URLs = "https://www.holistichempscotland.com/cbd-oils-pastes-capsules/"
try:
    Data_df = pd.concat([Data_df, scrape_Holistic_Hemp_Scotland(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Holistic_Hemp_Scotland(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Holistic_Hemp_Scotland(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in Holistic_Hemp_Scotland Scraper"],
                                                        "Product":["ERROR in Holistic_Hemp_Scotland Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Holistic_Hemp_Scotland")
num += 1
app.status = get_status(num)

### Hempura

In [50]:
def scrape_Hempura (URL):
    
    brand = "Hempura"
    country = "UK/EU"
    website = "hempura.shop"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("div", class_="desc")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("h1", class_="product_title")
        name = name.text.strip()
        
        
        
        #Get the Product description
        description = soup.find("div", id="tab-description").text.strip()
        description = description.replace('\n','')
        description = description.replace(" ","")
        
            
        #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
        price = soup.find("p", class_="price").text.strip()
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = price.replace(" ",".")
        price = re.findall(r"£(\d+\.\d+\s|\d+\s|\d+\.\d+)", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0]        
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       


        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    

        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]        

        #now to append the lists
        #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(converter("GBP", price))
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10
                brand_list.append(brand)
                price_list.append(converter("GBP", price))
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [51]:
URLs = "https://www.hempura.shop/cbd-oil/"
try:
    Data_df = pd.concat([Data_df, scrape_Hempura(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Hempura(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Hempura(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in Hempura Scraper"],
                                                        "Product":["ERROR in Hempura Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Hempura")

num += 1
app.status = get_status(num)

### MyCBD

In [52]:
def scrape_myCBD (URL):
    
    brand = "myCBD"
    country = "ES/EU"
    website = "mycbd.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("div", class_="cshere-woo-item-wrap")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("h3", class_="product_title")
        name = name.text.strip()
        
        
        
        #Get the Product description
        description = soup.find("div", id="tab-description")
        description = description.find("p").text.strip()
        description = description.replace('\n','')
        description = description.replace(" ","")
        
            
        #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
        price = soup.find("p", class_="price").text.strip()
        price = price.replace(" ",".")
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+)", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0]        
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       


        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    

        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]       

        #now to append the lists
        #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
        if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [53]:
URLs = "https://www.mycbd.com/de/produkt-kategorie%20/cbd-oel/"
try:
    Data_df = pd.concat([Data_df, scrape_myCBD(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_myCBD(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_myCBD(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in myCBD Scraper"],
                                                        "Product":["ERROR in myCBD Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_myCBD")
num += 1
app.status = get_status(num)

### deepnature project

In [54]:
def scrape_deepnature_project (URL):
    
    brand = "deepnature project"
    country = "AUS/EU"
    website = "deepnatureproject.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)
    
    driver = Edge(edge_path(), options = options)
    driver.get(URL)
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find("div", class_="products")
    link_basket = link_basket.find_all("li", class_="item")
    
    #Try if there is an accept cookies banner in the way, and click accept if that is the case
    try:
        driver.find_element_by_id("s-sv-bn") 
    except NoSuchElementException: #gets triggered if the cookie banner is not found
        a = 1
    else: #gets triggered if there is a cookie banner
        driver.find_element_by_id("s-sv-bn").click()
        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,"mfp-close")))
            driver.find_element_by_class_name("mfp-close").click()
        except NoSuchElementException: #gets triggered if the cookie banner is not found
            a = 1
       

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
         
        
        driver.get(link_list[i])
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,'price-box'))) #Somtimes the page doesn't load quickly enough, this is here to prevent that.   
        
        #Loop that can select both the 10ml and 30ml option
        stop = False
        stop_next = False
        while stop == False:
            
            volume_exists = False
            percent_exists = False
            mg_cbd_exists = False             
            
            if stop_next == True:
                stop = True
            
            try:    
                driver.find_element_by_css_selector(".swatch-option[aria-label='30 ml']") 
            except NoSuchElementException: #gets triggered if the last page is reached
                stop = True
            else: #gets triggered if there are more search pages and clicks on the next button
                stop_next = True
                #WebDriverWait(driver, 20,ignored_exceptions=StaleElementReferenceException).until(EC.presence_of_element_located((By.CSS_SELECTOR,".swatch-option[aria-label='30 ml']"))).click()
            
            if ((stop == True) & (stop_next == True)):
                volume_ml = 30.0
                volume_exists = True
            else:
                volume_ml = 10.0
                volume_exists = True
                
            
            soup = BeautifulSoup(driver.page_source,'lxml')
            #soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content)


            #Get the Name
            name = soup.find("h1", class_="page-title")
            name = name.text.strip() 
            
            if len(re.findall("Gutschein", name, re.IGNORECASE)) > 0:
                stop = True
                continue
            
            #Get the Brand            
            try:
                brand = "deepnature project - " + soup.find("div", itemprop="brand").text.strip()
            except:
                brand = "deepnature project - "

            #Get the Product description
            description = soup.find("div", class_="item-main").text.strip()
            description = description.replace('\n','')
            description = description.replace(" ","")
            description = re.search(r"Zutaten(.*?)Verzehrsempfehlung", description).group(1)


            #Get Price(s) if there are different products with different ml volumes we go to else, if there is only 1 product go to if
            price = soup.find("div", class_="price-box").text.strip()
            price = price.replace(" ",".")
            price = price.replace('\n','')
            price = price.replace(",",".")
            price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+)€", price)        
            price = [float(l) for l in price]
            price = sorted(price)
            price = price[0]        

            #Get percentage_cbd
            get = get_percent_cbd(name, description)
            percent_cbd = get[0]
            percent_exists = get[1]

            #Get the mg_cbd
            get = get_mg_cbd(name, description)
            mg_cbd = get[0]
            mg_cbd_exists = get[1]  

                  


            #now to append the lists
            #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)
            else:
                if mg_cbd_exists == True:
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)
                
            
            if stop_next == True:
                WebDriverWait(driver, 20,ignored_exceptions=StaleElementReferenceException).until(EC.presence_of_element_located((By.CSS_SELECTOR,".swatch-option[aria-label='30 ml']"))).click()
    
    driver.quit()
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [55]:
URLs = "https://www.deepnatureproject.com/de_INT/cbd-cbg-oele"
try:
    Data_df = pd.concat([Data_df, scrape_deepnature_project(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_deepnature_project(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_deepnature_project(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"],
                                                        "Company":["ERROR in deepnature_project Scraper"],
                                                        "Product":["ERROR in deepnature_project Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_deepnature_project")
            
num += 1
app.status = get_status(num)

### BlessedCBD (UK)

In [56]:
def scrape_BlessedCBD (URL):
    
    brand = "BlessedCBD"
    country = "UK"
    website = "blessedcbd.co.uk"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    products = soup.find_all("div", class_="slide")
        
    #go through all CBD Oil Products
    for i in range(len(products)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False  
        
        #Get the Product description
        description = products[i].find("div", class_="article__head")

        #Get the Name
        name = description.find("h4")
        name = name.text.strip()
                       
        
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]  

        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue            
        
        
        #Get Price(s) 
        price = description.find("strong")
        price = price.text.strip()
        price = price.replace(',','.')
        price = price.replace(' ','')
        price = price.replace('£','')
        price = float(price) 
        price = converter("GBP", price)
              
        
        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]

        #now to append the lists

        #print("single product")
        #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)
                      
                
           
        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df

In [57]:
URLs = "https://blessedcbd.co.uk/cbd-oils/"
try:
    Data_df = pd.concat([Data_df, scrape_BlessedCBD(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_BlessedCBD(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_BlessedCBD(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in BlessedCBD Scraper"],
                                                        "Product":["ERROR in BlessedCBD Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_BlessedCBD")
num += 1
app.status = get_status(num)

get_percent_cbd-single raises error:
'NoneType' object is not callable
get_volume_ml-single raises error:
'NoneType' object is not callable
get_percent_cbd-single raises error:
'NoneType' object is not callable
get_volume_ml-single raises error:
'NoneType' object is not callable
get_percent_cbd-single raises error:
'NoneType' object is not callable
get_volume_ml-single raises error:
'NoneType' object is not callable


### cbII (UK)

In [58]:
def scrape_cbII (URL):
    
    brand = "cbII"
    country = "UK"
    website = "cbii-cbd.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("li", class_="product-item")

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        
        #Get the Name
        name = soup.find("span", class_="base")
        name = name.text.strip()
        
        
        #Get the Product description
        description = soup.find("div", itemprop="description").text.strip()
        description = description.replace('\n','')
        description = description.replace(" ","")
        
        #Skip product if it contains capsules or balm or salve
        if skip_if_no_oil(name, description):
            #print("no oil = skip")
            continue
            
        #Get Price(s) and pick lowest one
        price = soup.find("div", class_="price-and-cta").text.strip()
        price = price.replace(" ","")
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = re.findall(r"£(\d+\.\d+\s|\d+\s|\d+\.\d+)", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0] 
        price = converter("GBP", price)
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       


        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    

        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]        
                
        #Find out if the title contains multiple products.
        multiples = get_multiples(name, description)
            
         #if there are multiples, set them for later division of endresult
        #print(multiples)
        if len(multiples) == 0:
            multiples = 1
        else:
            if all_equal(multiples):
                multiples = multiples[0]
            else: 
                continue
            
        #now to append the lists
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd*multiples)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10 * multiples
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [59]:
URLs = "https://www.cbii-cbd.com/shop/cbd-oil"
try:
    Data_df = pd.concat([Data_df, scrape_cbII(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_cbII(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_cbII(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in cbII Scraper"],
                                                        "Product":["ERROR in cbII Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_cbII")
num += 1
app.status = get_status(num)

### Holand and Barrett (UK,EU,INT)

In [60]:
def scrape_Holland_and_Barret (URL):
    
    brand = "Holland and Barret"
    country = "UK/EU/INT"
    website = "hollandandbarrett.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    products = soup.find_all("li", class_="gridColumn")
    
    #go through all CBD Oil Products
    for i in range(len(products)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(products)))
        brand = "Holland and Barret"
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False        
        product = products[i]
        
        
        #Get the Name
        try:
            name = product.find("h3", class_="title")
            name = name.text.strip()
        except:
            continue
        
        
        #Get the Product description
        description = name.replace('\n','')
        description = description.replace(" ","")
        
        #Skip product if it contains capsules or balm or salve
        if skip_if_no_oil(name, description):
            #print("no oil = skip")
            continue
        
        #Check for populare Brands
        if (len(re.findall("Grass&Co.", description, re.IGNORECASE)) != 0):
            brand = "Grass & Co."
        if (len(re.findall("HolisticHerb", description, re.IGNORECASE)) != 0):
            brand = "Holistic Herb"
        if (len(re.findall("JacobHooy", description, re.IGNORECASE)) != 0):
            brand = "Jacob Hooy"
        if (len(re.findall("Reakiro", description, re.IGNORECASE)) != 0):
            brand = "Reakiro"
        if (len(re.findall("Vitabiotics", description, re.IGNORECASE)) != 0):
            brand = "Vitabiotics"
        if (len(re.findall("AddCBD", description, re.IGNORECASE)) != 0):
            brand = "AddCBD"
        if (len(re.findall("LoveHemp", description, re.IGNORECASE)) != 0):
            brand = "Love Hemp"
            
        #Get Price(s) and pick lowest one
        price = product.find("h4", class_="price").text.strip()
        price = price.replace(" ","")
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = re.findall(r"£(\d+\.\d+\s|\d+\s|\d+\.\d+)", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0] 
        price = converter("GBP", price)
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]

        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       


        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    

        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1] 
   
                
        #Find out if the title contains multiple products.
        multiples = get_multiples(name, description)
            
         #if there are multiples, set them for later division of endresult
        #print(multiples)
        if len(multiples) == 0:
            multiples = 1
        else:
            if all_equal(multiples):
                multiples = multiples[0]
            else: 
                continue
            
        #now to append the lists
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd*multiples)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10 * multiples
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [61]:
URLs = "https://www.hollandandbarrett.com/shop/vitamins-supplements/cbd/?page=2&rpp=20&showAll=40&t=productformat_oil"
try:
    Data_df = pd.concat([Data_df, scrape_Holland_and_Barret(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Holland_and_Barret(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Holland_and_Barret(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Holland_and_Barret Scraper"],
                                                        "Product":["ERROR in Holland_and_Barret Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Holland_and_Barret")
num += 1
app.status = get_status(num)

### Medterra (UK)

In [62]:
def scrape_MedTerra (URL):
    
    brand = "MedTerra"
    country = "UK"
    website = "medterracbd.co.uk"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    products = soup.find_all("div", class_="ProductItem__Wrapper")
    
    #go through all CBD Oil Products
    for i in range(len(products)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(products)))
        brand = "MedTerra"
              
        product = products[i]
        
        
        #Get the Name
        try:
            name = product.find("h2", class_="ProductItem__Title")
            name = name.text.strip()
        except:
            continue
        
        option_s = product.find("div", class_="mt_featured_meta")
        option_s = option_s.find_all("option")
        
        for j in range(len(option_s)):   
            
            option = option_s[j]
            
            volume_exists = False
            percent_exists = False
            mg_cbd_exists = False  
            #Get the Product description
            description = option.text.strip()
            description = description.replace(" ","")

            #Skip product if it contains capsules or balm or salve
            if skip_if_no_oil(name, description):
                #print("no oil = skip")
                continue

            
            #Get Price(s) and pick lowest one
            price = description.replace(" ","")
            price = price.replace('\n','')
            price = price.replace(",",".")
            price = re.findall(r"£(\d+\.\d+\s|\d+\s|\d+\.\d+)", price)        
            price = [float(l) for l in price]
            price = sorted(price)
            price = price[0] 
            price = converter("GBP", price)

            #Get percentage_cbd
            get = get_percent_cbd(name, description)
            percent_cbd = get[0]
            percent_exists = get[1]

            #Get the mg_cbd
            get = get_mg_cbd(name, description)
            mg_cbd = get[0]
            mg_cbd_exists = get[1]       


            # If we have neither a percentage nor a mg value, we can not use the data
            if (mg_cbd_exists == False) & (percent_exists == False):
                #print("mg false and percent false")
                continue                    

            #Get the volume_ml
            get = get_volume_ml(name, description)
            volume_ml = get[0]
            volume_exists = get[1]        

            #Find out if the title contains multiple products.
            multiples = get_multiples(name, description)

             #if there are multiples, set them for later division of endresult
            #print(multiples)
            if len(multiples) == 0:
                multiples = 1
            else:
                if all_equal(multiples):
                    multiples = multiples[0]
                else: 
                    continue

            #now to append the lists
            if mg_cbd_exists == True:
                #print("viable")
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd*multiples)
                country_list.append(country)
                website_list.append(website)
                if volume_exists == True:
                    volume_list.append(volume_ml)
                else:
                    volume_list.append(np.nan)
            else:
                if ((volume_exists == True) & (percent_exists == True)):
                    #print("viable")
                    mg_cbd = volume_ml * percent_cbd * 10 * multiples
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [63]:
URLs = "https://medterracbd.co.uk/collections/cbd-oils"
try:
    Data_df = pd.concat([Data_df, scrape_MedTerra(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_MedTerra(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_MedTerra(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in MedTerra Scraper"],
                                                        "Product":["ERROR in MedTerra Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_MedTerra")
num += 1
app.status = get_status(num)

### CBD_FX (UK)

In [64]:
def scrape_CBD_FX (URL):
    
    brand = "CBD_FX"
    country = "UK"
    website = "cbdfx.co.uk"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    ignored_exceptions=(NoSuchElementException, StaleElementReferenceException)
    
    driver = Edge(edge_path(), options = options)
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("li", class_="grid__item")
     

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = "https://cbdfx.co.uk" + temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
         
        try:
            driver.get(link_list[i])
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,'product__price'))) #Somtimes the page doesn't load quickly enough, this is here to prevent that.   
            soup = BeautifulSoup(driver.page_source,'lxml')
           

            #Try if there is an accept cookies banner in the way, and click accept if that is the case
            try:
                driver.find_element_by_id("ba-cookie-dismiss") 
            except NoSuchElementException: #gets triggered if the cookie banner is not found
                a = 1
            else: #gets triggered if there is a cookie banner
                try:
                    #print("cookie dismissed")
                    driver.find_element_by_id("ba-cookie-dismiss").click()
                except:
                    a = 1

            option_s = soup.find("div", class_="prod_var")
            option_s = option_s.find_all("label")

            #iterate through the different options
            for j in range(len(option_s)):
                #print(j)
                option = option_s[j]
                #print(option)
                css = str(option)
                css = re.search(r"=(.*?)>", css).group(1)
                css = css.replace('"','')
                css ='label[for="'+css +'"]'
                #print(css)

                # This is an implicit generation of the xpath, because i couldn't figure out how to geneerate a proper xpath from the soup element "option".
                #xpath ="/html/body/div[4]/main/div[1]/div/div/div[2]/div[1]/form/div[1]/div/div/label[" + str(j+1) +"]"
                #print(xpath)
                #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).click()
                #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css))).click()
                soup = BeautifulSoup(driver.page_source,'lxml')
                #print(soup.find(css))
                driver.find_element_by_css_selector(css).click()
                WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME,'product__price')))
                #driver.find_element_by_id(id_).click()
                volume_exists = False
                percent_exists = False
                mg_cbd_exists = False             


                soup = BeautifulSoup(driver.page_source,'lxml')
                #soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content)


                #Get the Name
                name = soup.find("h1", class_="product-single__title")
                name = name.text.strip() 


                #Get the Product description
                description = soup.find("div", class_="product-details-tabs")
                description = description.find("div", id="tab-1").text.strip()
                description = description.replace('\n','')
                description = description.replace(" ","")

                if skip_if_no_oil(name, description) == True:
                    #print("skip no oil")
                    continue

                #Get Price(s) 
                price_s = soup.find("div", class_="product__price")
                price = price_s.find("div", class_="price__regular").text.strip()
                price = price.replace(" ","")
                price = price.replace('\n','')
                price = price.replace(",",".")
                price = re.findall(r"£(\d+\.\d+\s|\d+\s|\d+\.\d+)", price)        
                price = [float(l) for l in price]
                price = sorted(price)
                #there is a specific error on this website, where the sale price is always displayed (in the html, not on the site) as the lowest price of the first product
                #this if statement circumvents this
                if j == 0:
                    first_price = price[0]

                price_s = price_s.find("div", class_="price__sale").text.strip()
                price_s = price_s.replace(" ","")
                price_s = price_s.replace('\n','')
                price_s = price_s.replace(",",".")
                price_s = re.findall(r"£(\d+\.\d+\s|\d+\s|\d+\.\d+)", price_s)        
                price_s = [float(l) for l in price_s]
                price_s = sorted(price_s)
                if price_s[0] == first_price:
                    a = 1
                else: 
                    price.append(price_s)

                price = sorted(price)
                price = price[0]   
                price = converter("GBP",price)

                #Get percentage_cbd
                get = get_percent_cbd(name, description)
                percent_cbd = get[0]
                percent_exists = get[1]

                #Get the mg_cbd
                get = get_mg_cbd(option.text.strip(), description)
                mg_cbd = get[0]
                mg_cbd_exists = get[1] 

                # If we have neither a percentage nor a mg value, we can not use the data
                if (mg_cbd_exists == False) & (percent_exists == False):
                    #print("mg false and percent false")
                    continue 

                #Get the volume_ml
                get = get_volume_ml(name, description)
                volume_ml = get[0]
                volume_exists = get[1]

                #now to append the lists
                #print("volume=" + str(volume_exists) + "    percent="+str(percent_exists) + "    mg="+str(mg_cbd_exists))
                if ((volume_exists == True) & (percent_exists == True)):
                    #print("viable")
                    mg_cbd = volume_ml * percent_cbd * 10
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)
                else:
                    if mg_cbd_exists == True:
                        brand_list.append(brand)
                        price_list.append(price)
                        name_list.append(name)
                        cbd_list.append(mg_cbd)
                        if volume_exists == True:
                            volume_list.append(volume_ml)
                        else:
                            volume_list.append(np.nan) 
                        country_list.append(country)
                        website_list.append(website)
        except Exception as e:
            print("Error raised: " + str(e))
            continue
            



    driver.quit()
    
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [65]:
URLs = "https://cbdfx.co.uk/collections/cbd-tincture"
try:
    Data_df = pd.concat([Data_df, scrape_CBD_FX(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_CBD_FX(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_CBD_FX(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in CBD_FX Scraper"],
                                                        "Product":["ERROR in CBD_FX Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_CBD_FX")
num += 1
app.status = get_status(num)

### Pharma Premium Care (ES)

In [66]:
def scrape_Pharma_Premium_Care (URL):
    
    brand = "Pharma Premium Care"
    country = "ES"
    website =  "pharmapremiumcare.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("li", class_="product")
     

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("h3", class_="product-title")
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False
        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        #Get the Name
        
        name = soup.find("h1", class_="title-heading-left")
        name = name.text.strip()        
        
        
        #Get the Product description
        description = soup.find("div", class_="post-content").text.strip()
        description = description + soup.find("div", class_="fusion-text").text.strip()
        description = description + soup.find("div", class_="fusion-li-item-content").text.strip()
        description = description + soup.find("div", id="tab-additional_information").text.strip()
        description = description.replace('\n','')
        description = description.replace(" ","")
        
        
        #Skip product if it contains capsules or balm or salve
        if skip_if_no_oil(name, description):
            continue
        
                    
        #Get Price(s) and pick lowest one
        price = soup.find("div", class_="fusion-woo-price-tb").text.strip()
        price = price.replace(" ","")
        price = price.replace('\n','')
        price = price.replace(",",".")
        price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+)€", price)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0] 
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]
            
        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       
         
        
        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    
        
        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]      
        
        #print("MG: " + str(mg_cbd_exists) + "   ML: " + str(volume_exists) + "   %: " + str(percent_exists))    
        #now to append the lists
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10 
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [67]:
URLs = "https://www.pharmapremiumcare.com/tienda/?product_count=36"
try:
    Data_df = pd.concat([Data_df, scrape_Pharma_Premium_Care(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Pharma_Premium_Care(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Pharma_Premium_Care(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in CBD_FX Scraper"],
                                                        "Product":["ERROR in CBD_FX Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Pharma_Premium_Care")
num += 1
app.status = get_status(num)

### SunState Hemp (US)

In [68]:
def scrape_SunState_Hemp (URL):
    
    brand = "SunState Hemp"
    country = "USA"
    website = "sunstatehemp.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    product_s = soup.find_all("div", class_="Structure_Item")
    
    
    #go through all CBD Oil Products
    for i in range(len(product_s)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(product_s)))
        product = product_s[i]
        
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False
        
        #Get the Name        
        name = product.find("a", class_="Name")
        name = name.text.strip()        
        
        
        #Get the Product description
        description = name.replace('\n','')
        description = description.replace(" ","")
        
        
        #Skip product if it contains capsules or balm or salve
        if skip_if_no_oil(name, description):
            continue
        
                    
        #Get Price(s) and pick lowest one
        price = product.find("label", class_="Price").text.strip()
        price = price.replace(",",".")
        price = re.findall(r"(\d+\.\d+)", price, re.IGNORECASE)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0]
        price = converter("USD", price)
        
        #Get percentage_cbd
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]
            
        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       
         
        
        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    
        
        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]  
        
        #print("MG: " + str(mg_cbd_exists) + "   ML: " + str(volume_exists) + "   %: " + str(percent_exists))    
        #now to append the lists
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10 
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [69]:
URLs = "https://www.sunstatehemp.com/cbd-oil"
try:
    Data_df = pd.concat([Data_df, scrape_SunState_Hemp(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_SunState_Hemp(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_SunState_Hemp(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in SunState_Hemp Scraper"],
                                                        "Product":["ERROR in SunState_Hemp Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_SunState_Hemp")
num += 1
app.status = get_status(num)

### Aztec (UK)

In [70]:
def scrape_Aztec (URL):
    
    brand = "Aztec"
    country = "UK"
    website = "azteccbd.co.uk"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    product_s = soup.find_all("div", class_="best-product")
    
    
    #go through all CBD Oil Products
    for i in range(len(product_s)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(product_s)))
        product = product_s[i]
        
        volume_exists = False
        percent_exists = False
        mg_cbd_exists = False
        
        #Get the Name        
        name = product.find("p")
        name = name.text.strip()        
        
        
        #Get the Product description
        description = name.replace('\n','')
        description = description.replace(" ","")
        
        
        #Skip product if it contains capsules or balm or salve
        if skip_if_no_oil(name, description):
            continue
        
                    
        #Get Price(s) and pick lowest one
        price = product.find("p", class_="bt-price").text.strip()
        price = price.replace(",",".")
        price = re.findall(r"(\d+\.\d+)", price, re.IGNORECASE)        
        price = [float(l) for l in price]
        price = sorted(price)
        price = price[0]
        price = converter("GBP", price)
        
        #Get the percentage
        get = get_percent_cbd(name, description)
        percent_cbd = get[0]
        percent_exists = get[1]
            
        #Get the mg_cbd
        get = get_mg_cbd(name, description)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]       
         
        
        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue                    
        
        #Get the volume_ml
        get = get_volume_ml(name, description)
        volume_ml = get[0]
        volume_exists = get[1]     
        
        #print("MG: " + str(mg_cbd_exists) + "   ML: " + str(volume_exists) + "   %: " + str(percent_exists))    
        #now to append the lists
        if mg_cbd_exists == True:
            #print("viable")
            brand_list.append(brand)
            price_list.append(price)
            name_list.append(name)
            cbd_list.append(mg_cbd)
            country_list.append(country)
            website_list.append(website)
            if volume_exists == True:
                volume_list.append(volume_ml)
            else:
                volume_list.append(np.nan)
        else:
            if ((volume_exists == True) & (percent_exists == True)):
                #print("viable")
                mg_cbd = volume_ml * percent_cbd * 10 
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                volume_list.append(volume_ml)
                country_list.append(country)
                website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [71]:
URLs = "https://azteccbd.co.uk/product-category/shop-cbd-oil-drops/"
try:
    Data_df = pd.concat([Data_df, scrape_Aztec(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Aztec(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Aztec(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Aztec Scraper"],
                                                        "Product":["ERROR in Aztec Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Aztec")
num += 1
app.status = get_status(num)

### Alpinols (CH)

In [72]:
def scrape_Alpinols (URL):
    
    brand = "Alpinols"
    country = "CH/DE"
    website = "alpinols.com"
    country_list =[]
    website_list = []
    brand_list = []
    price_list = []
    name_list = []
    volume_list  = []
    cbd_list = []
    
    soup = BeautifulSoup(requests.get(URL, headers=headers).content,'lxml')
    link_basket = soup.find_all("div", class_="Grid__Cell")
     

    link_list = []
    for i in range(len(link_basket)):
        link_basket[i] = link_basket[i].find("a")
        temp_string = link_basket[i]
        temp_string = temp_string['href']
        temp_string = "https://alpinols.com" + temp_string
        link_list.append(temp_string)
    
    #go through all CBD Oil Products
    for i in range(len(link_list)):
        #print("----------------------------------------")
        #print("link:" + str(i+1) + "  of:" + str(len(link_list)))
        
        multiple_options = False
        
        soup = BeautifulSoup(requests.get(link_list[i], headers=headers).content,'lxml')
        
        #check if there is more than one product 
        test_multiple_options = soup.find_all("div", class_="ProductForm__Option")
        if len(test_multiple_options) != 0:
            multiple_options = True
            option_s = soup.find("div", class_="ProductForm__Variants")
            option_s = option_s.find("select")
            option_s = option_s.find_all("option")
        else:
            option_s = []
            
        #Get the Name    
        name = soup.find("h1", class_="ProductMeta__Title")
        name = name.text.strip()        
        #print(name)
        
        #Get the Product description
        if multiple_options == True:
            part_description = soup.find("div", class_="ProductMeta__Description")
            part_description = part_description.find("div", class_="Rte").text.strip()
            description = []     
            for j in range(len(option_s)):                           
                description.append(part_description + option_s[j].text.strip())
                description[j] = description[j].replace('\n','')
                description[j] = description[j].replace(" ","")
        else:
            description = soup.find("div", class_="ProductMeta__Description")
            description = description.find("div", class_="Rte").text.strip()
            description = description.replace('\n','')
            description = description.replace(" ","")

        
        #Skip product if it contains capsules or balm or salve
        if skip_if_no_oil(name, description, multiple_options = multiple_options):
            #print("no oil = skip")
            continue
        
                    
        #Get Price(s) and pick lowest one
        if multiple_options == True:
            price_s = []
            for j in range(len(option_s)):
                price = option_s[j].find("span", class_="money").text.strip()
                price = price.replace(" ","")
                price = price.replace('\n','')
                price = price.replace(",",".")
                price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+)€", price)        
                price = [float(l) for l in price]
                price = sorted(price)
                price = price[0] 
                price_s.append(price)
        else:
            price = soup.find("div", class_="ProductMeta__PriceList").text.strip()
            price = price.replace(" ","")
            price = price.replace('\n','')
            price = price.replace(",",".")
            price = re.findall(r"(\d+\.\d+\s|\d+\s|\d+\.\d+|\d+)€", price)        
            price = [float(l) for l in price]
            price = sorted(price)
            price = price[0] 
        
        #Get the percentage of CBD
        get = get_percent_cbd(name, description, multiple_options = multiple_options, option_s = option_s)
        percent_cbd = get[0]
        percent_exists = get[1]
            
        #Get the mg_cbd
        get = get_mg_cbd(name, description, multiple_options = multiple_options, option_s = option_s)
        mg_cbd = get[0]
        mg_cbd_exists = get[1]
        
         
        
        # If we have neither a percentage nor a mg value, we can not use the data
        if (mg_cbd_exists == False) & (percent_exists == False):
            #print("mg false and percent false")
            continue            
        
        
        #Get the volume_ml
        get = get_volume_ml(name, description, multiple_options = multiple_options, option_s = option_s)
        volume_ml = get[0]
        volume_exists = get[1]
            
           
        
        #print("MG: " + str(mg_cbd_exists) + "   ML: " + str(volume_exists) + "   %: " + str(percent_exists)) 
        #now to append the lists
        if multiple_options == True:
            for j in range(len(option_s)):
                if mg_cbd_exists == True:
                    #print("viable")
                    brand_list.append(brand)
                    price_list.append(price_s[j])
                    name_list.append(name)
                    cbd_list.append(mg_cbd[j])
                    country_list.append(country)
                    website_list.append(website)
                    if volume_exists == True:
                        volume_list.append(volume_ml[j])
                    else:
                        volume_list.append(np.nan)
                else:
                    if ((volume_exists == True) & (percent_exists == True)):
                        #print("viable")
                        mg_cbd = volume_ml[j] * percent_cbd[j] * 10 
                        brand_list.append(brand)
                        price_list.append(price_s[j])
                        name_list.append(name)
                        cbd_list.append(mg_cbd)
                        volume_list.append(volume_ml[j])
                        country_list.append(country)
                        website_list.append(website)
        else:
            if mg_cbd_exists == True:
                #print("viable")
                brand_list.append(brand)
                price_list.append(price)
                name_list.append(name)
                cbd_list.append(mg_cbd)
                country_list.append(country)
                website_list.append(website)
                if volume_exists == True:
                    volume_list.append(volume_ml)
                else:
                    volume_list.append(np.nan)
            else:
                if ((volume_exists == True) & (percent_exists == True)):
                    #print("viable")
                    mg_cbd = volume_ml * percent_cbd * 10 
                    brand_list.append(brand)
                    price_list.append(price)
                    name_list.append(name)
                    cbd_list.append(mg_cbd)
                    volume_list.append(volume_ml)
                    country_list.append(country)
                    website_list.append(website)

        
    data = {"Country":country_list,
            "Website":website_list, 
            "Company":brand_list,
            "Product":name_list,
            "Price[€]":price_list,
            "Volume[ml]":volume_list,
            "CBD[mg]":cbd_list}

    temp_df = pd.DataFrame(data)

    return temp_df
        

In [73]:
URLs = "https://alpinols.com/collections/cbd-oele"
try:
    Data_df = pd.concat([Data_df, scrape_Alpinols(URLs)]) #Call webscraper and merge results with already existing df  
    Data_df.reset_index()
except:
    try:
        Data_df = pd.concat([Data_df, scrape_Alpinols(URLs)]) #Call webscraper and merge results with already existing df  
        Data_df.reset_index()
    except:
        try:
            Data_df = pd.concat([Data_df, scrape_Alpinols(URLs)]) #Call webscraper and merge results with already existing df  
            Data_df.reset_index()
        except:
            print("ERROR")
            Data_df = pd.concat([Data_df, pd.DataFrame({"Country":["ERROR"], 
                                                        "Website":["ERROR"], 
                                                        "Company":["ERROR in Alpinols Scraper"],
                                                        "Product":["ERROR in Alpinols Scraper"],
                                                        "Price[€]":[np.nan],
                                                        "Volume[ml]":[np.nan],
                                                        "CBD[mg]":[np.nan]})])
            Data_df.reset_index()
            Error_list.append("scrape_Alpinols")
num += 1
app.status = get_status(num)

## Final Analysis and Export

### Final Analysis

In [74]:
#Data_df = finished_df

In [75]:
#Remove any duplicate rows from DataFrame
finished_df = Data_df
Data_df = Data_df.drop_duplicates(
  subset = ['Product', 'CBD[mg]'],
  keep = 'last').reset_index(drop = True)

#Remove erronous rows with less than 100mg cbd
Data_df = Data_df.rename(columns={"CBD[mg]": "cbd"})
Data_df = Data_df[Data_df.cbd >= 200]
Data_df = Data_df.rename(columns={"cbd": "CBD[mg]"})

#Calculate the €/mg and add as colum to Df
Euro_per_mg = Data_df["Price[€]"]/Data_df["CBD[mg]"]
Euro_per_mg = pd.DataFrame({"€/mg":Euro_per_mg})
Data_df = pd.concat([Data_df, Euro_per_mg], axis = 1)
Data_df.reset_index()

Data_df["€/mg"] = Data_df["€/mg"].apply(round_to_2)
working_df = Data_df
#calculate the mean
mean = np.mean(Data_df["€/mg"])

#Sort Values to get Cheapest Products
Cheapest10 = Data_df.sort_values("€/mg", ignore_index = True)
Cheapest10 = Cheapest10[Cheapest10.index<10]

#Create Final Format DataFrame
Data_df = pd.concat([pd.DataFrame({"Country":["↓10 Cheapest↓"],
                                    "Website":["↓10 Cheapest↓"],    
                                    "Company":["↓10 Cheapest↓"],
                                    "Product":["↓10 Cheapest↓"],
                                    "Price[€]":["↓10 Cheapest↓"],
                                    "Volume[ml]":["↓10 Cheapest↓"],
                                    "CBD[mg]":["↓10 Cheapest↓"],
                                    "€/mg":["↓10 Cheapest↓"]}), 
                     Cheapest10, 
                     pd.DataFrame({"Country":["-","-","↓COMPLETE LIST↓"],
                                   "Website":["-","-","↓COMPLETE LIST↓"],
                                   "Company":["-","-","↓COMPLETE LIST↓"],
                                   "Product":["-","-","↓COMPLETE LIST↓"],
                                   "Price[€]":["-","-","↓COMPLETE LIST↓"],
                                   "Volume[ml]":["-","-","↓COMPLETE LIST↓"],
                                   "CBD[mg]":["-","-","↓COMPLETE LIST↓"],
                                   "€/mg":["-","-","↓COMPLETE LIST↓"]}), 
                     Data_df])
Data_df.reset_index()

#Make subset dataframes with get_subset_df(df, ml, %)
ml10_percent5_df = get_subset_df(working_df, 10, 5)
ml10_percent10_df = get_subset_df(working_df, 10, 10)
ml10_percent15_df = get_subset_df(working_df, 10, 15)
ml10_percent20_df = get_subset_df(working_df, 10, 20)
ml10_percent25_df = get_subset_df(working_df, 10, 25)
ml30_percent5_df = get_subset_df(working_df, 30, 5)
ml30_percent10_df = get_subset_df(working_df, 30, 10)
ml30_percent15_df = get_subset_df(working_df, 30, 15)
ml30_percent20_df = get_subset_df(working_df, 30, 20)

#Extract Summaries from the dataframe and the subset dataframes
Summary_df = get_summary_df(working_df)

summary_ml10_percent5_df = get_summary_df(ml10_percent5_df)
summary_ml10_percent10_df = get_summary_df(ml10_percent10_df)
summary_ml10_percent15_df = get_summary_df(ml10_percent15_df)
summary_ml10_percent20_df = get_summary_df(ml10_percent20_df)
summary_ml10_percent25_df = get_summary_df(ml10_percent25_df)
summary_ml30_percent5_df = get_summary_df(ml30_percent5_df)
summary_ml30_percent10_df = get_summary_df(ml30_percent10_df)
summary_ml30_percent15_df = get_summary_df(ml30_percent15_df)
summary_ml30_percent20_df = get_summary_df(ml30_percent20_df)

### Export

In [77]:
#Get the path the programm is running in

#This one is for the actual programm    
#path = pathlib.Path().resolve().parents[0]
#this one is for the test version
path = pathlib.Path().resolve()



# datetime object containing current date and time
now = datetime.now()

# dd-mm-YY__H-M-S
now = now.strftime("%d-%m-%Y__%H-%M-%S")

#create the output path
output_path = str(path) + "\\" + str(now) + ".xlsx"

#Create document
writer = pd.ExcelWriter(output_path, engine='openpyxl')

Summary_df.to_excel(writer, sheet_name='Total_Summary')
Data_df.to_excel(writer, sheet_name='Raw_data')
summary_ml10_percent5_df.to_excel(writer, sheet_name='10ml 5%')
summary_ml10_percent10_df.to_excel(writer, sheet_name='10ml 10%')
summary_ml10_percent15_df.to_excel(writer, sheet_name='10ml 15%')
summary_ml10_percent20_df.to_excel(writer, sheet_name='10ml 20%')
summary_ml10_percent25_df.to_excel(writer, sheet_name='10ml 25%')
summary_ml30_percent5_df.to_excel(writer, sheet_name='30ml 5%')
summary_ml30_percent10_df.to_excel(writer, sheet_name='30ml 10%')
summary_ml30_percent15_df.to_excel(writer, sheet_name='30ml 15%')
summary_ml30_percent20_df.to_excel(writer, sheet_name='30ml 20%')

ml10_percent5_df.to_excel(writer, sheet_name='10ml 5% data')
ml10_percent10_df.to_excel(writer, sheet_name='10ml 10% data')
ml10_percent15_df.to_excel(writer, sheet_name='10ml 15% data')
ml10_percent20_df.to_excel(writer, sheet_name='10ml 20% data')
ml10_percent25_df.to_excel(writer, sheet_name='10ml 25% data')
ml30_percent5_df.to_excel(writer, sheet_name='30ml 5% data')
ml30_percent10_df.to_excel(writer, sheet_name='30ml 10% data')
ml30_percent15_df.to_excel(writer, sheet_name='30ml 15% data')
ml30_percent20_df.to_excel(writer, sheet_name='30ml 20% data')


writer.save()

num += 1
app.status = 100

In [78]:
end = time.time()
print(f"Runtime of the program is {end - start}")
print(str(len(Error_list)) + " Errors were raised")
print("by:")
print(Error_list)

Runtime of the program is 808.6394288539886
0 Errors were raised
by:
[]
